In [1]:
# install ktrain on Google Colab
!pip3 install ktrain

     |████████████████████████████████| 25.2MB 121kB/s 
     |████████████████████████████████| 421.8MB 22kB/s 
     |████████████████████████████████| 6.7MB 42.9MB/s 
     |████████████████████████████████| 983kB 37.7MB/s 
     |████████████████████████████████| 245kB 39.3MB/s 
     |████████████████████████████████| 757kB 36.6MB/s 
     |████████████████████████████████| 471kB 33.8MB/s 
     |████████████████████████████████| 3.9MB 33.8MB/s 
     |████████████████████████████████| 450kB 41.8MB/s 
     |████████████████████████████████| 1.1MB 35.6MB/s 
     |████████████████████████████████| 3.0MB 36.9MB/s 
     |████████████████████████████████| 890kB 39.0MB/s 
  Created wheel for ktrain: filename=ktrain-0.17.3-cp36-none-any.whl size=25249269 sha256=27c2c73878a63ee2d17e255a381ddfb51edfa9160c58300dc3239b9f12a96966
  Stored in directory: /root/.cache/pip/wheels/24/87/78/8dcbc028acc22bc106e03a954f0de2f02e57fb42438ecb177d
  Created wheel for keras-bert: filename=keras_bert-0.84.0-cp36-no

In [10]:
# import ktrain and the ktrain.text modules
import ktrain
from ktrain import text

In [11]:
ktrain.__version__

'0.17.3'

# Multiclass Text Classification Using BERT and Keras
In this example, we will use ***ktrain*** ([a lightweight wrapper around Keras](https://github.com/amaiya/ktrain)) to build a model using the dataset employed in the **scikit-learn** tutorial: [Working with Text Data](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html).  As in the tutorial, we will sample 4 newsgroups to create a relatively small multiclass text classification dataset.  The objective is to accurately classify each document into one of these four newsgroup topic categories.  This will provide us an opportunity to see **BERT** in action on a relatively smaller training set.  Let's fetch the [20newsgroups dataset ](http://qwone.com/~jason/20Newsgroups/) using scikit-learn.

In [12]:
#!wget 'final_dataframe.csv'

In [13]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/consultant_desc_tags.csv', encoding = 'latin')
print(df)

       Unnamed: 0  ...                                               tags
0               0  ...  Personaladministration,FI: Finanzbuchhaltung,Z...
1               1  ...  Personaladministration,FI: Finanzbuchhaltung,Z...
2               2  ...  Personaladministration,FI: Finanzbuchhaltung,Z...
3               3  ...  Personaladministration,FI: Finanzbuchhaltung,Z...
4               4  ...  Personaladministration,FI: Finanzbuchhaltung,Z...
...           ...  ...                                                ...
25208       25208  ...  Anforderungsmanagement,CSS,HTML,JavaScript,Pro...
25209       25209  ...  Anforderungsmanagement,CSS,HTML,JavaScript,Pro...
25210       25210  ...  Anforderungsmanagement,CSS,HTML,JavaScript,Pro...
25211       25211  ...  Anforderungsmanagement,CSS,HTML,JavaScript,Pro...
25212       25212  ...  Anforderungsmanagement,CSS,HTML,JavaScript,Pro...

[25213 rows x 7 columns]


In [14]:
# fetch the dataset using scikit-learn
categories = ['Bildungsabschlüsse', '(Ausl) vgl. Dipl.-Ing.', 'Fremdsprachen', 'Albanisch', 'Arabisch', 'Armenisch', 'Bahasa Malaysia / Indonesia', 'Bulgarisch', 'Chinesisch', 'Deutsch', 'Dänisch', 'Englisch', 'Finnisch', 'Französisch', 'Griechisch (Neu-)', 'Hebräisch', 'Hindi', 'Iranisch', 'Italienisch', 'Japanisch', 'Koreanisch', 'Latein', 'Luxemburgisch', 'Niederländisch', 'Norwegisch', 'Polnisch', 'Portugiesisch', 'Rumänisch', 'Russisch', 'Schwedisch', 'Serbo-Kroatisch', 'Slowakisch', 'Slowenisch', 'Spanisch', 'Tschechisch', 'Türkisch', 'Ungarisch', 'Vietnamesisch', 'Methoden & Verfahren', 'Administration', 'Angebotslegung / Vertrieb', 'Einführung SAP', 'Einführung Standardsoftware', 'Entwicklung', 'Juristische Kenntnisse', 'Konstruktion', 'Org.-/Managementmethoden', 'Qualitätssicherung', 'Sonstige Methoden', 'Strategieberatung', 'Durchführung vonTrainings', 'Admin Tools', 'Akquise', 'Analysetechniken', 'Anforderungsmanagement', 'Angebotserstellung', 'Anwendungsentwicklung', 'Arbeitsplatzanalysen', 'Arbeitsrecht', 'Architekturen', 'Archiv / DMS', 'Aspektorientierte Methoden', 'Assessment', 'Aufbau-/Ablauforg. Techniken', 'Auftrag', 'Ausgliederung', 'Backup-/Recovery Lösungen', 'Banking', 'Basis', 'Betriebssystem', 'Betriebssystem', 'Business Intelligence', 'Business Rules Mgt. System', 'CAD', 'CAM', 'CASE-Werkzeuge', 'CM/CRM', 'Callcentertechnologien', 'Cards+Payment 2.5G', 'Cards+Payment 3.G', 'Chancen-/Risikenanalyse', 'Change-Management', 'Contentmanagement', 'Contentmanagement', 'DMS', 'DMS', 'DV-Training', 'Data Warehouse', 'Data Warehouse methodisch', 'Datenbank', 'Datenbank', 'Datenbank Management', 'Datenbankzugriffstechnologien', 'Datenmodellierung methodisch', 'Datenmodellierungstools', 'Datenqualität', 'Digitale Betriebsprüfung', 'Digitale Signatur', 'ETL-Werkzeuge', 'EURO-Konvertierung', 'Eigenentwicklung Steria Mummert', 'Einführungsmanagement', 'Entwicklungsumgebung', 'Erhebungstechniken', 'Evolutionäre Algorithmen', 'Finanzdienstleister', 'BI-Frontend', 'Funktionen', 'Generierung Helpdateien', 'Geographische Informationssysteme', 'Geschäftsprozessoptimierung', 'Gesellschaftsrecht', 'Groupware', 'HW-/SW-Auswahl', 'IT-Betrieb', 'IT-Management', 'Immobilien', 'Industrial Solution', 'Installation', 'Instandhaltung', 'Insurance', 'Integration', 'Integration', 'Integration', 'Internet/Intranet', 'Investitionsmanagement', 'Java Application Server', 'Kartellrecht', 'Kommunikation', 'Kommunikation', 'Kommunikation', 'Komponentenorientierte SW-Entwicklung', 'Kreativitätstechniken', 'Kundenservice', 'Künstliche Intelligenz', 'Logistik', 'Mail', 'Materialwirtschaft', 'Mathematisch-statist. Methoden', 'Methoden-Training', 'Middleware Produkte/Kommunikationssoftware', 'Migration', 'Mikroprozessor Karte', 'Mobile', 'Mobile Technologies', 'Netztechnologien', 'Netzwerk', 'Oberfläche', 'Objektorientierte Methoden', 'Operations Research', 'Org.entwicklung/-beratung', 'Outsourcing Geschäftsprozesse', 'Outsourcing IT-Prozesse', 'PDM', 'Performance Management', 'Personalbemessung', 'Personalmanagement', 'Personalwirtschaft', 'Portaltechnologie', 'Problem Management', 'Produktion', 'Programmiersprache', 'Projektmanagement', 'Projektmanagement Modelle', 'Protokolle', 'Prozessmodellierung', 'Prozessmodellierung methodisch', 'Public', 'Qualitätsmanagement', 'Qualitätssicherung', 'Rechenzentrum', 'Rechnungswesen', 'Ressourcenverwaltung', 'Review / Audit', 'SAP R/2', 'SAP R/3', 'SSW TELCO-BSS', 'SSW TELCO-OSS & Security', 'SW Configuration Management', 'Security', 'Service Desk/Monitoringtools', 'Shop Systeme', 'Smartcards', 'Software Schätzverfahren', 'Software-Entwicklung methodisch', 'Software-Ergonomie', 'Sonstige Standardsoftware', 'Sonstige Strategieberatung', 'Standardsoftware E-Business', 'Standardsoftware Elektron. Zahlungsverkehr', 'Standardsoftware Health', 'Standardsoftware Public', 'Stellenbewertung', 'Steuerung (betriebl.)', 'Steuerungsmodelle', 'Test', 'Testdatengenerierung', 'User Interface Design', 'Vertragsrecht', 'Vertrieb', 'Vertriebslogistik', 'Verzeichnisdienste', 'Wirkungsforschung/-analyse', 'Wirtschaftlichkeitsuntersuchungen', 'Workflow', 'mySAP.com Produkte', 'Öffentliches Recht', '.NET Compact Framework', '.NET Framework', '.NET Mobile Internet Toolkit', '.NET Remoting', '.NET Remoting', '3COM', 'ABACUS (Meldewesen)', 'ABAKUS (Inv.Bank)', 'ABAP/4', 'ABC-Analyse', 'ABC-Analyse Kundenportfolios', 'ABCD-Analyse', 'ADA', 'ADABAS', 'ADABAS', 'ADAPT Standards', 'ADK', 'ADO', 'ADO.NET', 'ADPS', 'ADS-Online', 'ADS/Win 2000', 'ADS/Win 2000', 'ADW', 'AFS Andrew File Service', 'AGB-Recht', 'AIX', 'AIX', 'ALK-GIAP', 'FS-CRM AO', 'ANSI.X12', 'ANSI.X12', 'APL', 'APO-DP: Demand Planning', 'APO-DP: Demand Planning', 'APO-GATP: Global Availabil. Check', 'APO-GATP: Global Availabil. Check', 'APO-LiveCache', 'APO-LiveCache', 'APO-PP/DS: Prod.Plan/Detail.Sched.', 'APO-PP/DS: Prod.Plan/Detail.Sched.', 'APO-SNP: Supply Network Planning', 'APO-SNP: Supply Network Planning', 'APO-Stammdaten und Integration', 'APO-TPVS: Transportation Planning', 'APO-TLB: Transport Load Builder', 'APPC von IBM', 'ARC-INFO', 'ARIS 6', 'ARIS Toolset', 'ARIS Toolset', 'ARS Remedy Applications', 'ARS', 'ASP Application Service Providing', 'ASP Application Service Providing', 'ASP.NET', 'ASYS3', 'ATL', 'ATM', 'AWK', 'Abrechnung', 'Accelerated SAP', 'MS Access', 'Evidian (Access Master)', 'Access Master PKI', 'Acta', 'Active Server Pages', 'Active X', 'Actuate E-Business-Suite', 'Adobe GoLive', 'Adonis', 'Aladin', 'Altersvorsorge', 'Altova XML Spy Integr.Dev.Environm.', 'Altova XML Spy XSLT Editor', 'Amipro', 'Analytische Zeitschätzung', 'Anbieterauswahl', 'Anforderungsanalyse', 'Anforderungskatalog', 'Angebots-Projektleitung', 'Angebotsbewertung', 'Angebotsmanagement', 'Angebotspräsentation', 'Angebotsverhandlung', 'Angebotsvorlage', 'Anreizsysteme', 'Ant', 'Anti Virus', 'Apache Webserver', 'Apache Xerces XML Parses (DOM)', 'Apache Xerces XML Parses (SAX)', 'Aperture', 'Applets / Servlets', 'Application Flow Design', 'Applikationsarchitektur', 'Arbeitgeberleistungen', 'Arbeitsvertragsrecht', 'Arbeitszeitmodelle', 'Arbor Software / Essbase', 'ArcServ', 'ArchiveLink', 'Arcplan', 'Ardent/Informix', 'ArgoUML', 'Artemis', 'Artemis', 'IBM Ascential DataStage', 'Aspect', 'Aspektorientierte Analyse', 'Aspektorientierte Entwicklung', 'Aspektorientiertes Design', 'Assembler', 'Assessment-Center', 'Asset Product./.gif/.jpg/Flash...', 'Auditierung nach IT-Grundschutz', 'Auditorausbildung', 'Aufgabenanalyse', 'Aus- und Fortbildung', 'Aus- und Weiterbildung', 'Ausschreibung', 'Auswahl', 'Auswahl DQ-Tools', 'Automatic Cash Management System/Montran', 'AxiOSS', 'BAAN 4GL-Sprachen', 'BANYAN', 'BB3 / bank 21', 'ABAP Objects', 'BC: ALE BAPI', 'BC: Berechtigungskonzepte', 'BC: Business Connector', 'BC: CATT (Computer Aided Test Tool)', 'BC: Development Workbench (ABAP/4)', 'BC: Formularprogrammierung', 'BC: IDOC', 'BC: ITS', 'BC: KTW Korrektur-/Transp.wesen', 'BC: Schnittstellenentwicklung', 'BC: Security', 'BC: Workflow', 'BC: externe Printmanager (RDI)', 'BCA Bank Customer Accounting', 'BEA WebLogic Integration', 'BEA WebLogic Integration', 'BEA WebLogic Integration', 'BGP', 'BPM', 'BPO-Engine', 'BPO-Engine/TRAMAG', 'BS2000', 'BS2000', 'BSP Trade Business System Planning', 'BUS Betreuerunterstützg. Firmenkd.', 'Baan', 'Bachman', 'Bachman', 'Backupexe', 'Balanced Scorecard', 'BA-BA IAS', 'BA-CRA Basel II', 'BA-FDB', 'BA-RR Meldewesen', 'Bankvertragsrecht', 'BanqPro WP-Abwicklung', 'Basis', 'Bea WebLogic', 'Bea WebLogic Enterprise WLE', 'Bea WebLogic Server WLS', 'Bea WebLogic Server WLS', 'Beantragungs-/Genehmigungsverf.', 'Benchmarking/vergleich. Berichtswes.', 'Berechtigung', 'Berichtswesen', 'Bestandsbetreuung', 'Beteiligungsrecht', 'Betrieb/Technik/Produktion', 'Betriebskonzeption', 'Betriebsorganisation', 'Betriebsverfassungsrecht', 'Betriebswirtschaft/KLR/Controll.', 'Betriebsübergang', 'Authentifizierung - Biometrie', 'Black Belt Training', 'Blaze Advisor', 'Bloomberg', 'Bluetooth', 'Bonaparte', 'Bonaparte', 'Bootstrap', 'Bootstrap', 'Borland Delphi', 'Borland IDE', 'Brainstorming', 'Break-Even auf Kapitalwertbasis', 'Brokat', 'Budgetierung', 'Bugzilla', 'Business Objects', 'Business Objects: BusinessObjects', 'BusinessObjects Web Intelligence', 'Business Rules Approach', 'Business Rules Mgt. Method.', 'Business-Plan', 'C', 'C#', 'C++', 'C++ - Builder', 'CA-Datacom', 'CA-Librarian', 'CA-Roscoe', 'CA-Unicenter/TNG', 'CA-Unicenter/TNG', 'CA-Unicenter/TNG', 'CAPI', 'CATIA', 'CATS Cross Application Time Sheet', 'CC Evaluierung nach CEM', 'CC:Mail', 'CDE', 'CFM: Corporate Finance Management', 'CHILL', 'CICS', 'CICS', 'CISCO', 'CISCO (PIX)', 'CML Loans', 'CMS (Sicherheitenverwaltung)', 'CO-OM-ABC (PKR)', 'CO-OM', 'CO-PA (ER/MSR)', 'CO-PC-CS (Produktkostencontrolling)', 'CO-OM-OPA (Gemeinkosten-auftr.)', 'COBOL', 'COBOL II', 'COCOMO-II-Methode', 'COCOMO-Methode', 'COM', 'COM+', 'COM+', 'CORBA', 'CORBA', 'CORBA 2.3', 'CORBA 3', 'AFL (Leasing)', 'CRM- und Vertriebsstrategie', 'SAP CRM CIC', 'CRM: Custom. Intact. Cent./Se/Sa/Mar.', 'SAP CRM Datenmodell (Mapping IS-U)', 'CRM: Internet Sales/Serv./Marktg.', 'SAP CRM Mobile Sales/Service/Marketing', 'CRM: Mobile Sales/Serv./Marktg.', 'CS', 'CSP', 'CVS', 'CYGWIN', 'CYGWIN', 'Caché', 'Caché', 'Capability Maturity Mod. Intgr.', 'Capability Maturity Mod. Intgr.', 'Case 4', 'Case/4/0', 'Case/4/0 der Firma Microtool', 'Centura', 'Ceyoniq-Server', 'Ceyoniq-Server', 'Change-Management als Tool', 'Changemanagement', 'Cisco IOS', 'Citrix Metaframe', 'Citrix Metaframe', 'Citrix Metaframe', 'Clarify', 'ClearCase', 'ClearCase', 'ClearQuest', 'Client / Server User Interface Design', 'Clusteranalyse', 'Coaching', 'Coaching', 'Code-Analyse', 'Cognos', 'Cognos', 'Cognos Analytical Applications', 'Cognos Impromptu', 'Cognos Impromptu', 'Cognos Powerplay', 'Cognos Powerplay', 'Coldfusion', 'Compuware-FileAid/CS', 'Microfocus QADirector', 'Microfocus QARun', 'Connect:Direct', 'Connected', 'Continuus', 
              'Contr.+Audit f.Inf.+Relat.Tech.', 'Controller Exp.Leadersh.Worksh.', 'Cool:Gen', 'Cool:Gen', 'Corporate Identity', 'CrossWorlds', 'CrossWorlds', 'CrossWorlds Advanc. Cross Refer.', 'CrossWorlds Advanc. Mapping', 'Cryptography', 'Crystal Reports', 'Culprit', 'DART', 'DART', 'DB2', 'DB2', 'DB2 Connect', 'DB2-Parallelprozessverarbeitung', 'DB2-Parallelprozessverarbeitung', 'DB2-fragmentierte Datenbestände', 'DB2-fragmentierte Datenbestände', 'DB2/2', 'DB2/2', 'DB2/SQL', 'DB2/SQL', 'DCI', 'DCOM', 'DEC Ultrix', 'DEC Ultrix', 'DM Deposits AM', 'DMADV Einf.Prozesse/Produkte', 'DNS Domain Name System', 'DOS/VM', 'DOS/VM', 'DPG Schnittstelle', 'DPPX', 'DPPX', 'DQM/Profiles', 'DXF', 'Darlehen', 'Data Mining', 'Kundenbindung (CHURN)', 'Frühwarnsysteme/Vorausb.', 'Mißbrauchs-Erkennung (FRAUD)', 'Optimierung Marketing Kampagne', 'Segmentierung', 'Umsatzprognose', 'IBM DataStage', 'Data-Point-Methode', 'Datenbank-Haushaltsanalyse', 'Datenqualität', 'Datenschutz', 'Datenschutz', 'Datenschutzrecht', 'Datenträgeraustausch', 'Datenträgeraustausch', 'Datenvalidierung', 'Datex P', 'Dbase', 'Dbase', 'Decalog', 'Decnet', 'Delphi', 'Delta Alea', 'Dienstleistungsvertragsrecht', 'Digital Archive', 'Digital Archive', 'DirX (Siemens)', 'Dispositive Polizeisysteme', 'Doc-to-Help 6.0', 'Documentum', 'Documentum', 'Documentum WebPublisher', 'Dokumenten-Management DCS', 'Dokumenterstellungssystem DES', 'Domino.doc', 'Dreamweaver', 'Dynamic Host Configuration Protocol', 'DÜVA', 'DÜVA-A', 'E-Commerce-Strategie', 'Epiphany/Information', 'EAI', 'EC-CS-CSF (Konsolidierung, Entw.)', 'EC-PCA (Profit Center-R)', 'EDI', 'EDIFACT', 'EDIFACT', 'EDIFACT', 'EERM', 'EIFFEL', 'EJB', 'EJB', 'EJB 1.1', 'EJB 2.0', 'EJB-Container', 'EMC Centera', 'EMC Centera', 'EMV', 'ENUM', 'E-R Modeler', 'ERM', 'ERM', 'ERwin', 'ETI', 'ETI', 'EURO', 'EURO-Konverter', 'EURO-Toolset', 'EVIDIAN', 'EVIDIAN  OM4T', 'Earned Value', 'Eastman Kodak', 'EasyArchiv', 'EasyArchiv', 'Easytrieve Plus', 'Eclipse', 'Eclipse Rich Client Platform (ERCP)', 'Eclipse Rich Client Platform (ERCP)', 'Effizienz-/Kosten-Nutzenanalyse', 'Einsatz von Tools (SPSS u.a.)', 'Elektronische Aktenführung', 'Elektronische Aktenführung', 'Elektronisches Vertragsmgt.', 'Elektronisches Vertragsmgt.', 'Embedded SQL', 'Endevor', 'Enterprise Architect', 'Enterprisearchitektur', 'Entlohnungs-/Tarifsysteme', 'Entrust', 'Ereignis-Prozessketten', 'Ereignis-Prozeßketten', 'Eskalationsverfahren', 'Ethernet v.a. 10 GE', 'Ethernet-Switching-Technol.', 'Ethernet-Technologien', 'Evaluierung Outsourcing', 'Evaluierung Outsourcing', 'Exchange Infrastructure-XI', 'EC-EIS (Unternehmens-Informationssystem)', 'Expect', 'Expertensysteme', 'Extraction Transf. Loading/ETL', 'FDDI', 'FI-AA: Anlagenbuchhaltung', 'FI-AP: Kreditorenbuchhaltung', 'FI-AR: Debitorenbuchhaltung', 'FI-CA', 'FI-GL: Hauptbuchhaltung', 'FI-SL', 'FI: Finanzbuchhaltung', 'FIDELITY', 'FILEAID2', 'FMEA Failure Mode Eff. Analysis', 'FORTRAN', 'FOXPRO', 'FS-BP (Geschäftspartner)', 'FS-BP: Financial  Services Business Partner', 'FS-CD (Inkasso/Exkasso)', 'FS-CD (Inkasso/Exkasso)', 'FS-CM (Schaden)', 'FS-ICM (Provisionen)', 'FS-ICM', 'FS-PM (Bestand)', 'FS-RI (Rückversicherung)', 'FTA Failure Tree Analysis', 'Fachliche QS Angebot', 'Fernabsatzvertragsrecht', 'File Transfer Protokoll TCP/iP', 'File Transfer Protokoll TCP/iP', 'FileNet', 'FileNet', 'FileNet Government für D-A-CH', 'FileNet Government für D-A-CH', 'Finanzen/Rechnungswesen', 'Firewalls', 'Netzwerk - Firewall', 'Fischgrätenanalyse', 'ForeHelp Premier 2001', 'Formsprache', 'Fragebogen', 'FreeBSD', 'FreeBSD', 'Front Arena', 'Frontpage', 'Frühwarnsystem', 'Function-Point-Methode', 'Funktionenmodellierung mit SA', 'Fusions-/Übernahmerecht', 'Fusionsberatung', 'Führen im Projekt', 'Führungsfunktionen (auf Zeit)', 'Führungsstil', 'GALA', 'GAS-X', 'GCOS7', 'GCOS7', 'GDPdU Grundsätze Datenzugriff', 'GEOS WP-Abwicklung', 'GL Trade', 'GP Geschäftspartner', 'GPM', 'GPO-/Ablauftools', 'GPRS', 'GRID Management', 'GSM', 'GSM-Gateway', 'GSTOOL', 'BSI (GSTOOL)', 'Gemeinkostenwertanalyse', 'Genesys', 'Genesys', 'Genesys', 'Genetische Algorithmen', 'Genetische Programmierung', 'Geno-Light', 'Genocash', 'GeoMedia', 'Gewährleistungsrecht', 'Großprojekterfahrung', 'Großprojektmanagement', 'Gründung Projekt-Office', 'Guardian 90', 'Guardian 90', 'Gupta SQL Windows', 'Gupta SQLBase', 'Gupta SQLBase', 'HACCP Hazard An.Crit.Con.Point', 'HBCI (Home Banking Computer Interface)', 'HDLC', 'HH-Steuerung/Konsolidierung', 'HOAI', 'HP OpenView', 'HP OpenView', 'HP-UX', 'HP-UX', 'HP-UX 11.22. auf IA64 (Itanium)', 'HP-UX 11.22. auf IA64 (Itanium)', 'HP-UX 11i Mission Crit.Env.', 'HP-UX 11i Mission Crit.Env.', 'HPS', 'HSCSD', 'HSQLDB', 'HSQLDB', 'HTML', 'HTML', 'HTML Help', 'HTTP', 'HTTPS', 'HVX', 'HVX', 'Haftungsrecht', 'Haushaltsrecht', 'Helppartner 6.1', 'Hibernate', 'Himalaya', 'Hochv.HP-UX-Cluster/MCServGuard', 'Hochv.HP-UX-Cluster/MCServGuard', 'Hochverfügbarkeit/Ausfallcluster', 'Hochverfügbarkeit/Ausfallcluster', 'Hotline / Anwenderbetreuung', 'Hub & Spoke', 'Hummingbird', 'Hummingbird', 'Hyper Ware', 'Hyperion Essb.OLAP-S.WinNT/Unix', 'Hyperion Integration Server', 'Hyperwave', 'MARK Modulare Anwendung Real Kredite', 'IBM Content Manager', 'IBM Content Manager', 'IBM DB2 OLAP-Server:WinNT/Unix', 'IBM FIX Engine', 'IBM FIX Engine', 'IBM Process Choreographer', 'IBM Process Choreographer', 'IBM Process Choreographer', 'IBM WBI Server Foundation', 'IBM WBI Server Foundation', 'IBM/Intelligent und Text Miner', 'ICIS', 'IDMS', 'IDMS', 'IDS2', 'IDS2', 'IEW', 'IEW', 'IGES', 'IGS Internet Graphics Server', 'IIOP', 'ILOG Business Rules Mgt. System', 'IM Investitionsmanagement', 'MM-IM: Inventory Management', 'IMAP', 'IMS/DB', 'IMS/DB', 'IMS/DC', 'IMS/DC', 'IMS/DC', 'INFORMIX', 'INFORMIX', 'INFORMIX 4GL', 'INFORMIX SQL', 'INGRES', 'INGRES', 'IP Mask', 'IPSec', 'IPX / ODI', 'IS-Banking', 'IS-CS: Cable Solution', 'IS-Environment / Health & Safety', 'IS-H: Hospital', 'IS-H: med', 'IS-PP', 'IS-R: Retail', 'IS-T: Telecommunication', 'IS-U: Utility', 'ISA Dialogmanager', 'ISDN', 'ISO 15022', 'ISO 15022', 'ISO 9000/2002', 'ISPF - IBM Host', 'IT-Basiswissen/VCT (M+P-intern)', 'IT-Governance', 'IT-Grundschutzhandbuch', 'IT-Grundschutz', 'IT-Konzeption', 'IT-Kostenmanagement', 'IT-Sicherheit', 'IT-Strategie', 'IAM - Identity Management', 'Imaging / el. Archivierung', 'Imaging / el. Archivierung', 'Imbus', 'Inasys Portfolio-Management System', 'Income Process Designer', 'Individualarbeitsrecht', 'Individuelle Migrationen', 'Infinity', 'Inform. Techn. Infrastr. Library', 'Informatica', 'Ingres 4GL', 'Inno setup', 'Innovation Leadership', 'Innovator', 'Innovator', 'Installshield Admin Studio', 'Installshield Editor', 'InterShop 4', 'Internet Information Server', 'Internet Secure Systems (ISS)', 'Interview', 'Netzwerk - Intrusion Detection / Prevention', 'Intrusion Prevention', 'Intuitiv', 'Intuitiv', 'Investitionsmanagement', 'Ishikawa-Diagramm', 'Istaufnahme/-analyse', 'J++', 'J2EE', 'J2EE', 'J2ME', 'JAVA', 'JAVA', 'JAVA 2', 'JAVA 5', 'JBoss', 'JBoss', 'JBuilder', 'JBuilder', 'JBuilder', 'JCL', 'JDBC', 'JDBC', 'JDO JavaDataObject', 'JDO JavaDataObject', 'JDeveloper', 'JFreeChart', 'JIRA', 'JNDI', 'JProbe', 'JProfiler', 'JUNIT', 'Jahr-2000-Problematik', 'Jahresschätzung', 'JasperReports', 'Java', 'Java Cards', 'Java Database Connectivity', 'Java Server Faces', 'Java Server Pages', 'Java VisualCafé', 'Java beans', 'JavaScript', 'JavaWebStart', 'KARA', 'KAVIA', 'KDBD Commerzbank log. Dat.modell.', 'KDE', 'KLR/Controlling', 'KORDOBA', 'KTQ-Verfahren Kliniken', 'Kalkulation', 'Kaltakquise/Erstgespräch', 'Karriereplanung/Führungsberatung', 'Karten Betriebssysteme', 'Kennzahlen', 'Kennzahlensystematik', 'Key', 'Key-Accountplanung', 'Key-Analyse', 'Knowledge Wave', 'Kombi.Ganzzahl.Heurist.z.Optim.', 'Kommunikationsstrategie/-planung', 'Kompass', 'Kondor+ Handelssystem Reuters', 'Konfigurationsmgt. Konzepte', 'Konsolidierung', 'Kontraktmanagement', 'Konzernbereiche', 'Kostensenkung', 'Kreditwesengesetz', 'Verschlüsselung - Kryptographische Algorithmen', 'Verschlüsselung - Kryptoprozessoren', 'Kundenbefragung', 'LAN-Manager', 'LDAP-Protokoll', 'LDAP-Protokoll', 'LDAP-Server', 'LDAP-Server', 'LEGACY', 'LISP', 'LU 6.2', 'Laufzettel', 'Laufzettel', 'Leitbild', 'Leitstandssysteme Feuerwehr', 'Leitstandssysteme Polizei', 'Leu-Smart', 'Lineare Programmierung', 'Linux', 'Linux', 'Lizenzverträge/Urheberrecht', 'Load Balancing', 'Load Balancing', 'Logistik', 'Lotus 123', 'Lotus 123', 'Lotus Notes', 'Lotus Notes', 'Lotus Notes Mail', 'Lotus Script', 'ME: Mobile Engine', 'MFC', 'MIG 21', 'Kerberos', 'MKS Source Integrity', 'MKS Source Integrity', 'MM-Dien: Dienstleistungsabwicklung', 'MM-INV: Bestandsführung', 'MM-PUR: Einkauf', 'MM-SRV: Dienstleistungsabwicklung', 'MM-WM: Lagerverwaltung', 'MQ Series Workflow (ehem. FlowMark)', 'MQ Series Workflow (ehem. FlowMark)', 'MS Access', 'MS Access', 'MS Access XP', 'MS Access XP', 'MS Active Directory', 'MS Analysis Services', 'MS Analysis Services', 'MS Analysis Services', 'MS Application Center', 'MS Business Solutions Navision', 'MS Developer Studio', 'MS Excel Makroprg. VisualBasic', 'MS Internet Information Server', 'MS Outlook', 'MS Outlook', 'MS SQL DB-Design', 'MS SQL DB-Design', 'MS SQL Server', 'MS SQL Server', 'MS SQL-Server Admin.', 'MS SQL-Server Admin.', 'MS SQL-Server Analys.Serv.', 'MS SQL-Server Analys.Serv.', 'MS SQL-Server Perf.Tuning', 'MS SQL-Server Perf.Tuning', 'MS Visual C++', 'MS Visual Studio', 'MS-DOS', 'MS-DOS', 'MS-Excel', 'MS-Excel', 'MS-Exchange', 'MS Exchange', 'MS Mail', 'MS-Office', 'MS-Project', 'MS-Project', 'MS-Word', 'MTM', 'MVS', 'MVS', 'MVS', 'Mach', 'Macintosh', 'Macintosh', 'Macromedia JRun', 'Maestro', 'Magic Service Desk', 'Performance Management', 'Mantis', 'Marketing Manager', 'Marketing/Vertrieb', 'Maverick Framework', 'Megs', 'Memo', 'Merant', 'Mercator', 'Mercator', 'Mercator', 'Mercury InteractiveTestdirector', 'Merva', 'MetaSuite', 'Metadatenmodellierung',
              'Metaframe 1.8', 'Metaframe XP', 'Metaframe XP', 'Metaphase', 'Methode / Konzept', 'Metrica-Performance Manager', 'Metrica/NPR', 'Micro Focus Dialogsystem', 'MicroStrategy', 'Microfocus Cobol Workbench', 'Microfocus Net Express (Merant)', 'Micrologica', 'Micromuse', 'Micromuse', 'MS .NET', 'Microsoft BizTalk', 'Microsoft BizTalk', 'Microsoft BizTalk', 'MS Content Management Server', 'MS Explorer', 'MS Installer', 'Microsoft SharePoint', 'Microsoft SharePoint', 'MS Systems Management Server', 'MS Transaction Server', 'MS Visual Interdev', 'Migration', 'Migration', 'Migration Finanz- u. Rechnungswesen', 'Migration Logistik', 'Migration Personalwirtschaft', 'Migration: Toolset', 'Migrationswerkzeuge', 'Mindmap', 'Minerva/Carleton', 'Mitarbeiterbefragung', 'Mitarbeiterbeurteilung', 'Mitarbeiterzufriedenheit', 'Mitbestimmungsrecht', 'Mobile Portal', 'Moderation', 'Moderation', 'Modula2', 'Motivation', 'Multicash', 'Multicast-/Broadcastverfahren', 'Multimomentstudie', 'Multimomentstudie', 'Multiprojektmanagement', 'Murex Handelssystem', 'MySQL', 'MySQL', 'Netzwerk - Network Address Translation (NAT)', 'NATURAL', 'NCR Teradata', 'NETBIOS', 'NETBIOS mit Samba', 'NGOSS / OSS-Java', 'NLP-Training', 'NOVELL', 'NOVELL NetWare', 'NOVELL NetWare', 'NPS: Network Product. System', 'NPS: Network Product. System', 'NSE', 'Nagios', 'Natural Construct', 'Natural Engineering Workbench', 'NetBSD', 'NetBSD', 'NetBeans', 'Netbackup', 'Netlife Brokerage Systems', 'Netscape Application Server', 'Netscape Browser', 'Netscape Server', 'Netscape/Sun iPlanet', 'Netwall', 'Networker', 'Netzwerk-Sicherheit', 'Netzwerk-Topologie', 'Netzwerkoptimierungen', 'Neuronale Netze', 'Nexus', 'Nichtlineare Programmierung', 'NonStopSQL', 'NonStopSQL', 'Novell Groupwise', 'Novell (Identity Manager DirXML)', 'Novell Netware 5.x', 'Novell Netware 5.x', 'Novell Netware 6.x', 'Novell OES Linux', 'Novell OES Linux', 'Novell OES Netware', 'Novell V4', 'Novell ZENworks', 'Novell eDirectory 8.x', 'Nutzwertanalyse', 'OBS Online Banken System', 'ODBC', 'ODBC', 'ODBC', 'OLEDB', 'OLEDB', 'OMT', 'OO-Analyse', 'OO-Design', 'OO-Programmierung', 'OOA', 'OOD', 'OQL Object Query Language', 'ORACLE 6', 'ORACLE 6', 'ORACLE 8i', 'ORACLE 8i', 'ORACLE CRM', 'ORACLE CRM', 'ORACLE V7', 'ORACLE V7', 'ORACLE: Oracle DBMS', 'ORACLE: Oracle DBMS', 'ORBIS Krankenhausinfosystem', 'OS/2', 'OS/2', 'OS/2 Presentation Manager', 'OS/390 (ehem. MVS)', 'OS/390 (ehem. MVS)', 'OS/400', 'OS/400', 'OSF/Motif', 'OSI', 'OSPF', 'OVV', 'Object Engineering Process', 'Object-Point-Methode', 'Objektorient. Anforderungsanalyse', 'Objektrelationales Mapping', 'Occam 2', 'Odette', 'Office Vision', 'Open EJB', 'Open LDAP', 'Open Master', 'Open Text', 'Open Text', 'OpenBSD', 'OpenBSD', 'OpenMaster', 'OpenShop', 'OpenUi', 'OpenUi', 'Oracle', 'Oracle', 'Oracle 10g', 'Oracle 10g', 'Oracle Application Server', 'Oracle Application Server', 'Oracle Collaboration Suite', 'Oracle Collaboration Suite', 'Oracle DB Administration', 'Oracle DB Performance Tuning', 'Oracle Designer 2000', 'Oracle Designer 9i', 'Oracle Designer/Developer 2000', 'Oracle Developer 2000', 'Oracle Discoverer Administrat.', 'Oracle Discoverer Desktop/Plus', 'Oracle Discoverer10g', 'Oracle Discoverer9i', 'Oracle Enterprise Manager', 'Oracle Financials', 'Oracle Forms', 'Oracle Internet Directory', 'Oracle PC Express', 'Oracle PL/SQL', 'Oracle Performance Tuning', 'Oracle Performance Tuning', 'Oracle Project', 'Oracle Real Appl. Cluster/RAC', 'Oracle Real Appl. Cluster/RAC', 'Oracle SQL*FORMS', 'Oracle Web Server', 'Oracle applications', 'Oracle9i', 'Oracle9i', 'Orbix2000', 'Organis. Messe/Veranstaltung', 'Organisationsmanagement', 'Outsourcing', 'Outsourcing Management', 'Outsourcing Management', 'Outsourcing V3 Betrieb', 'PABA/Q', 'PAMStorage (H&S)', 'PANVALET', 'PBS archive add ons', 'PE Payment Solution', 'PHP', 'PKI', 'Zertifikatsmanagement /Public Key Infrastructure (PKI)', 'PL/1', 'PLM: Product Lifecycle Management', 'PM: Instandhaltung', 'PM: Plant Maintenance', 'PORTAL', 'PP-BC: Verwaltung techn. Grunddaten', 'PP-DE: Prozeßindustrie', 'PP-MRP: Disposition', 'PP-PI: PP for Process Industries', 'PP-SFC: Produktionssteuerung', 'PP-SOP:Absatzplanung', 'PRISM', 'PRO/E', 'PS: Project System', 'PSCD Collection & Disbursement', 'PSM Public Sector Management', 'PSRM Record Management', 'PVCS', 'PVCS Version Manager', 'Package Konzeption', 'Panvalet', 'Paradigm Plus', 'Paradox - Borland', 'Paradox - Borland', 'Pascal', 'Patrol', 'Pavone Espresso', 'Perl', 'Pega Rules', 'Pega Rules Process Commander', 'PeopleSoft', 'Perform.kalk. mit ESTIMATOR', 'Performance Management Konzepte', 'Performance Messtools', 'Performanceanalyse/-prognose', 'Performanceplanung', 'Perl', 'Personaladministration', 'Personalbeschaffung', 'Personalcontrolling', 'Personaleinsatzplanung', 'Personalentwicklung', 'Personalentwicklung', 'Personalkostenplanung', 'Personalplanung', 'Petri-Netze', 'PiroNet PiroBase', 'PiroNet PiroBase', 'Planung Projektverlauf', 'Planungstools', 'Plumtree', 'Team-Point', 'Portale', 'Portfolio Analyse', 'Portfoliomanagement', 'Portia Portfolio-Management System', 'Poseidon for UML', 'PostgreSQL', 'PostgreSQL', 'PowerCenter/PowerMart', 'Powerbuilder', 'Powerbuilder', 'PowerDesigner', 'Powerdesigner', 'Powerpoint', 'Powertier', 'Predict', 'Predict-Case von SAG', 'Primavera', 'Prince2', 'Private/Public Key Infrastructure', 'Privatisierung', 'ProFiskal', 'Problem Management Konzepte', 'Problem Management Tools', 'Problemmanagement', 'Process Improvement', 'Prodom (System der Sparkassen)', 'Produkt-/Kd.gruppenkalkulation', 'Produkt-Semantik', 'Produkthaushalt', 'Professional Homepage Tools', 'Profit-Center', 'Projekt-Qualitätssicherung', 'Projektmanagement', 'Projektplanung/-steuerung', 'Projektpriorisierung', 'Protos', 'Prozedurale Anwendungsentwicklung', 'Prozess-/Ablaufbeschreibung', 'Prozess-Dokumentation', 'Prozessanalyse', 'Prozesskostenrechnung', 'Prozessportale', 'Präsentation', 'Public Key', 'Public Relations/Presse+Öffentl.', 'QM-Modelle', 'QM-Systeme', 'QMF', 'QMF', 'QT', 'QUMAS e-DocCompliance', 'QUMAS e-DocCompliance', 'Qualitätscontrolling', 'Qualitätsinstrumente', 'Qualitätsplanung', 'Qualitätssicherung', 'Qualitätszirkel', 'RA: Anlagenbuchhaltung', 'Microsoft (RAS, ...)', 'RCS Revision Control System', 'RE (Real Estate)', 'RE (Real Estate)', 'RE-FX (Flächenmanagement)', 'REXX', 'RF: Finanzbuchhaltung', 'RK: Kostenrechnung', 'RMI', 'RMI', 'RPC', 'RPG II', 'RPM', 'RTGS+', 'RTGS+', 'RUP', 'RaQuest', 'Radius System', 'Rapid Appl. Development', 'Rational RequisitePro', 'Rational Rose', 'Rational Rose', 'Rational Unified Process', 'Rational XDE', 'Rationalisierungsberatung', 'RealTime', 'Rechner Verbund System VW', 'Rechner Verbund System VW', 'Rechts-und Betriebsformen', 'Rechtsformwahl', 'Referenzschätzung', 'Regression/Korrelation', 'Reisekosten', 'Reisemanagement', 'Rekrutierung', 'Remedy Applications', 'Remedy Applications', 'Remote Method Invocation', 'Remote Update Verfahren', 'Remote Update Verfahren', 'Ressourcenmanagement', 'Reuters', 'Reviewdurchführung', 'Robohelp Office 2002', 'Rochade', 'S-Designor', 'S1 CRM', 'SA', 'SAMBA', 'SAMBA-Sprache (Aktionen/Bed.)', 'SAP Business Information Warehouse', 'SAP Business Information Warehouse', 'SAP Business Information Warehouse', 'SAP Business Workflow', 'SAP CRM', 'SAP DVS', 'SAP Enterprise Portal', 'SAP R/3 Anwenderschulungen', 'SAP Solution Manager', 'SAP Solution Manager', 'SAP-Mail', 'SAPScript', 'SAS', 'SAS AF / EIS / Enterprise Reporter', 'SAS Access', 'SAS Base / Macro', 'SAS Clients: Enterprise Guide/ Enterprise Reporter', 'SAS Connect', 'SAS Enterprise Miner', 'SAS Internet plus AppdevStudio', 'SAS Warehouse Administrator', 'SAS multidim. Datenbank MDDB', 'SCCS Source Code Control System', 'SCM: Suppy Chain Management', 'SCO Unix', 'SCO Unix', 'SD', 'SD-SH: Versandabwicklung', 'SD-SM: Verkaufsabwicklung', 'SD: Bestellabwicklung', 'SD: Sales and Distribution', 'SDF2', 'SECCOS', 'SED', 'SEM Banking Profit Analyzer', 'SEM Banking Risk Analyzer', 'SEM Banking Strategy Analyzer', 'SEM-BCS Business Consol. Syst.', 'SEM-BPS Business Plan. and Sim.', 'SEM-CPM Corporate Perform. Mgt.', 'SEMPA', 'SEM: Strategic Enterprise Mgt.', 'SERM', 'SESAM', 'SESAM', 'SINIX', 'SINIX', 'SM: Service Management', 'SMALLTALK', 'SMALLWORLD', 'Cellent FS', 'SML', 'SMS', 'SMTP', 'SNA', 'SNA', 'SNIFF', 'SNMP', 'SNMP', 'SOA', 'SOAP', 'SOLARIS', 'SOLARIS', 'SPA', 'SPSS/Clementine', 'SPUFI', 'SPUFI', 'SQL', 'SQL Navigator (Quest)', 'SQL Navigator (Quest)', 'SQL-Anywhere Watcom', 'SQL-Anywhere Watcom', 'SQL-DS', 'SQL-DS', 'SQL-Server', 'SQL-Server', 'SQLJ', 'SQS-Tools', 'SRM: Suppl. Relationsh. Man. EBP 3.0', 'SS7', 'SSL-Client Authentication', 'SSL-Infrastruktur', 'SSL-Infrastruktur', 'SUBITO', 'SUMMIT-D', 'SUN ONE Directory Service', 'SUN OS', 'SUN OS', 'SWOT-Analyse', 'SWT', 'SWT/JFace', 'Sagent Solution', 'Sagent Solution', 'Sanierungskonzepte', 'Saperion', 'Saperion', 'Satzungsrecht', 'Schadenaktenanalyse', 'Schichtplanung', 'Schlüsselmanagement', 'Scout2 (Delta)', 'Screen Design', 'Seagate', 'Secunet', 'SSL/TSL (Secure Socket Layer)', 'Secure network e-mail', 'SeeBeyond', 'SeeBeyond', 'SeeBeyond', 'Selbstaufschreibung', 'Selbstaufschreibung', 'Semantisches Projektmodell', 'Seminare zu Steuerungsmodelle', 'Sensitivitätsanalysen', 'Service Level Agreements', 'Service Level Management', 'Set-Top-Box', 'Shell-Scripts (Unix)', 'Sich.konz.n.BSI Sich.handbuch', 'Sicherheitskonz. n. 7799 Meth.', 'Sicherheitskonz. n. CC Meth.', 'Sicherheitskonz. n.BSI GSHB', 'Sicherheitstechnologien', 'Siebel', 'Siebel Analytics', 'Signaturgesetz', 'Skill Radar', 'Silverstream', 'IAM - Single Sign On', 'Tonbeller', 'Six Sigma', 'Six Sigma', 'SnagView', 'Software-/ Cognitive-Ergonomie', 'Softwareergonomie', 'Softwarevertragsrecht', 'Solide Edge', 'Solide Works', 'Sozialrecht', 'Spice', 'Spring', 'Staffware', 'Starmoney', 'Stellen-/Funktionsbeschreibung', 'Stellenwirtschaft', 'Step', 'Steuerrecht', 
              'Steuerungsunterstützung', 'Stichprobenverfahren', 'Straigt Through Processing', 'Strateg. Geschäftsfeldentwicklung', 'Strategiekonzept', 'Struts - Open Source Framework', 'Subversion', 'Summit Handelssystem', 'Sun One', 'Sun Workshop', 'Sunopsis', 'Supra', 'Swift', 'Swift', 'Swift Fin', 'Swift Net Services', 'Swift/Cash-Management', 'Sybase', 'Sybase', 'Sybase', 'Sybase EAS /Jaguar CTS', 'Sybase Power Designer', 'Sybase PowerJ', 'Systems Enginer', 'Systems Security', 'TARGET', 'TARGET/TARGET2', 'TCL', 'TCP/IP', 'TCP/IP', 'TDS', 'TEBIS', 'TELON', 'TIBCO', 'TIBCO', 'TIBCO', 'TIBCO Rendezvous', 'TIBCO SmartSockets', 'TLS', 'TOAD', 'TP8', 'TPS Labs', 'TR-TM-MM (Geldhandel)', 'TRAMAG', 'TRAMAG', 'TSL/TQL', 'TSO', 'TSO Oberfläche', 'TUXEDO', 'Tandem Novistop', 'Tandem Novistop', 'Tarantella Server', 'Tarantella Server', 'Target Costing', 'Tarifrecht', 'Tcl / TK', 'Tcl / TK', 'Team Improvement', 'Teamwork', 'Techniken/Vorgehensweise', 'Technologie-Assessment', 'Teledienstegesetz', 'Telefon. Kundenbefragung', 'Telelogic Doors', 'Tendermanagement', 'Teradata', 'Teradata', 'Testautomatisierung', 'Testfall-Methode', 'Testfallbeschreibung', 'Testgetriebene Entwicklung', 'Testkonzept/-handbuch', 'Testplanung', 'Testschlüssel BluePages', 'Testtools', 'Themenevaluation', 'Tivoli', 'Tivoli', 'Tobit David', 'Together', 'TogetherJ', 'Tomcat', 'Toplink', 'Total Data Quality Management', 'Train the Trainer', 'Tributum Capital Gain', 'Tributum First Tax', 'True64', 'True64', 'Trust Center', 'Trust Services', 'UDDI', 'UDS', 'UML', 'UML Notation', 'UMTS', 'UMTS USIM', 'UNIX', 'UNIX', 'UPnP', 'Unicad / MicroStation', 'Unified Software Development Process', 'Unternehmensführung', 'Update.com', 'Use-Case-Modellierung', 'Use-Case-Modellierung', 'User Interaction Analysis', 'Usermanagement', 'Identity Access Management (IAM)', 'V-Modell', 'V-Modell', 'V-Modell XT', 'V-Modell XT', 'V3.Fonds', 'VBA', 'VDA/FS', 'VISUAL C++', 'VJ++', 'VM/CMS von IBM', 'VM/CMS von IBM', 'VMS', 'VMS', 'VModell97', 'VMware ESX', 'VMware ESX', 'VMware ESX', 'VMware GSX', 'VMware GSX', 'VMware GSX', 'VMware Hochverfügbarkeitscluster', 'VMware Hochverfügbarkeitscluster', 'VMware Virtual Center', 'VMware Workstation', 'VMware Workstation', 'VMware Workstation', 'VP/MS', 'VPN', 'Netzwerk - Virtual Private Network (VPN)', 'VSAM', 'VSE', 'VSE', 'VT4Java', 'Validity Data Cleansing', 'Vantive', 'Veranstaltungsmanagement', 'Verfahrensanweisungen', 'Verfassungsrecht', 'Vergaberecht', 'Vergütungsmanagement', 'Verrechnungspreisermittlung', 'Verschlüsselungsverfahren', 'Versicherungsstatistik CD-ROM', 'Vertragsrechtl. QS Angebot', 'Verwaltung', 'Verwaltungsrecht', 'Vignette', 'Vignette StoryServer', 'Systemsicherheit - Virenschutz/Anti-Malware', 'VisiBroker', 'Visio', 'Visual Age', 'Visual Basic', 'Visual Basic 1.1', 'Visual Basic .Net', 'Visual Basic Script', 'Visual Basic Script', 'Visual Studio.NET', 'MS Visual Workbench', 'VisualAge Generator', 'VisualAge for Java', 'VisualWorks', 'Visualinfo', 'Vitria', 'VoIP', 'Vorgesetzteneinschätzung', 'MS Vss', 'WAP', 'SAP NetWeaver Application Server', 'SAP NetWeaver Application Server', 'WBI InterChange Server', 'WBI Message Broker', 'WBI Message Broker', 'WBI Message Broker', 'WEB-SSO', 'WINDOWS', 'WINDOWS', 'WINDOWS NT', 'WINDOWS NT', 'WINDOWS NT', 'WINDOWS XP', 'WINDOWS XP', 'WLAN', 'WML', 'WML', 'WP2', 'WSDL', 'Watcom C++', 'Web Content Management', 'Web Content Management', 'Web Design', 'Web Design', 'Web Publishing', 'Web Publishing', 'Web Service-Architekturen', 'WebDAV', 'WebGain', 'IBM WebSphere', 'WebSphere Applicat. Server WAS', 'WebSphere Business Integration', 'WebSphere Business Integration', 'WebSphere Business Integration', 'MQSeries (Websphere MQ)', 'WebSphere MQ Integrator Broker', 'WebSphere MQ Integrator Broker', 'WebSphere MQ Integrator Broker', 'WebSphere MQ Workflow', 'WebSphere MQ Workflow', 'WebSphere MQ Workflow', 'WebSphere Process Server', 'WebSphere Process Server', 'WebSphere Process Server', 'WebSphere Studio Appl. Devel.', 'Werkvertragsrecht', 'Wertanalyse', 'Wettbewerbsrecht', 'WiKi MoinMoin', 'Windows 2000', 'Windows 2000', 'Windows 2000 Client', 'Windows 2000 Client', 'Windows 2000 Server', 'Windows 2000 Server', 'Windows 2000 mit Cluster Serv.', 'Windows 2003 Server', 'Windows 2003 Server', 'Windows CE', 'Windows CE', 'Windows Forms', 'Windows NT Client', 'Windows NT Client', 'Windows NT Hochverfüg.-Cluster', 'Windows NT Server', 'Windows NT Server', 'Windows Vista', 'Windows.Vista', 'Wireless Local Area Networks', 'Wirtschaftliche QS Angebot', 'Wissensmanagement', 'Wordpro', 'Workflow-Management-System', 'Workflow-Management-System', 'Workshop (Konzeption / Durchführung)', 'X.25', 'X.509', 'X/Motif', 'XML', 'XML Schema (XSD)', 'XML Signature', 'XML-Formatting Objects', 'XML-Transformationen', 'XML-Verschlüsselung', 'XPDITER', 'ZV-Light', 'Zeitaufnahme', 'Zeitmessung', 'Zeitwirtschaft', 'Zertifizierungen', 'Zim (4GL)', 'Zim (4GL)', 'Zugriffslisten', 'Zulassungsverfahren ZW Expert', 'ccc harvest', 'domino.workflow (ehem. ProZessware)', 'e-Recruiting', 'iXos', 'iXos', 'objectiF', 'Verschlüsselung - single DES', 'Verschlüsselung - triple DES', 'webMethods', 'webMethods', 'webMethods', 'winsure', 'winsure bAV', 'zOS', 'zOS', 'Geschäftsprozesse', 'Automotive', 'Banking Finance', 'Branchenübergreifend', 'Bundes-/Landesverwaltung', 'Fernsehen', 'Health Care', 'Hochschulen', 'Hörfunk', 'Industrie / Handel', 'Kommunalverwaltung', 'Logistics', 'Pharma/Chemie', 'Printmedien', 'Sicherheit', 'Sonstige öffentl. Einrichtungen', 'Sozialverwaltung', 'Telekommunikation', 'Transport', 'Utilities', 'Versicherungsarchitektur', 'Abfall/Stadtentwässerung/Stadtreinig.', 'Arbeitsschutz/-medizin', 'Arbeitsverwaltung', 'Asset Management', 'Automation/Überwachung', 'Automobilhersteller', 'Automobilzulieferer', 'BSS', 'Bahn', 'Bauen / Verkehr / Wohnen', 'Beschaffungs-/Bestandsmgt.', 'Betriebsprozesse', 'Betriebsprozesse', 'Betriebsprozesse', 'Bundespolizei/Zoll', 'Bundeswehr', 'Controlling', 'Diskrete Produktionssysteme', 'Dispositionssysteme', 'Einzelhandel', 'Endkundenabrechnung', 'Endkundenservice', 'Energieerzeugung/-förderung', 'Energiehandel', 'Energienahe Dienstleister', 'Energietransport/-verteilung', 'Enhanced Services', 'Entsorgung', 'Europäische Kommission', 'Evangelische Landeskirche', 'Fahrwegüberwachung', 'Festnetz-Betriebsprozesse', 'Festnetz-Services', 'Feuerwehr', 'Filialbankorganisation', 'Finanzen', 'Finanzen/Haushalt', 'Finanzverwaltung', 'Forschungseinrichtungen', 'Gasversorger', 'Gesetzliche Krankenversicherung', 'Gesetzliche Rentenversicherung', 'Gesetzliche Unfallversicherung', 'Gesundheit/Soziales/Jugend/Sport', 'Großhandel', 'Grundstoffindustrie Chemie', 'Grünflächen/Landschaft/Friedhof', 'Hauptamt/Serviceunterstützung', 'Hoch-/Tiefbau/Vermessung', 'Identitätsmanagement', 'Immobilien', 'Industrie (allgemein)', 'Industrie-Sektoren', 'Innenverwaltung', 'Internet-Services', 'Investmentbanking', 'IuK/TuI', 'Justizverwaltung', 'KFZ-Versicherung', 'Kommunale Betriebe', 'Kompositversicherung Industrie', 'Kompositversicherung Privat', 'Konstruktionsbüros', 'Kraft-/Wärmeversorger', 'Kraftfahrt-Bundesamt', 'Krankenhäuser / Kliniken', 'Krankenkassen (Gesetzl.)', 'Kredit', 'Kultus/Schule', 'Kultus/Schule', 'Kundenmanagement', 'Labor/Werkstatt', 'Landwirtschaft/Ernährung/Forsten', 'Lehre/Forschung', 'Liegenschaften/Gebäudewirtschaft', 'Marketing', 'Mineralölkonzerne', 'Ministerial-/Behördenverwalt. (übergr.)', 'Mittelinstanz', 'Mobile-Datenkommunikation', 'Mobilfunk-Betriebsprozesse', 'Mobilfunk-Services', 'Monitoring und Control', 'OSS', 'Objektschutz', 'Core Banking', 'Personalmanagement', 'Personalverwaltung', 'Pharmahandel', 'Pharmahersteller', 'Polizei', 'Post', 'Private Krankenversicherung', 'Private Unfallversicherung', 'Produktionsprozesse', 'Prozesssysteme', 'Recht/Sicherheit/Ordnung', 'Rechtsschutz', 'Reisendeninformation', 'Rettungswesen', 'Rückversicherung', 'Sachvers.-Gewerbe/Industrie', 'Sachvers.-Privat', 'Sonstige Geschäftsprozesse', 'Spezielle Prozesse', 'Strassenverkehr', 'Stromversorger', 'Technischer Handel', 'Umwelt / Verbraucherschutz', 'Umwelt / Verbraucherschutz', 'Unternehmenssteuerung', 'Versicherungsanstalten/-ämter', 'Versorgungsverwaltung', 'Vertrieb', 'Vertriebslinie / Regiebereich', 'Vertriebsmanagement', 'Verwaltung', 'Videomanagement', 'Vorsorge', 'Wasserversorger', 'Wertpapier', 'Wirtschaft/Technologie', 'Wirtschafts-/Beschäftigungsförderung', 'Wohlfahrtsverbände', 'Wohnungsbaugesellschaften', 'Zahlungsverkehr', 'Zubereiter Chemie', 'private Sicherheitsdienste', 'Ämter/Anstalten', 'ÖPNV', 'Abrechnung', 'Abrechnung', 'Abrechnungen/Ersatzansprüche', 'Aktivierung', 'Aktivierung', 'Aktivierung', 'Aktivierung', 'Aktivierung', 'Allianzen', 'Anlagekontenverwaltung', 'Anlagemanagement', 'Anlagenbuchhaltung', 'Anlagenplanung', 'Anlieferung', 'Apotheke', 'Applikationen', 'Arbeitgeber', 'Asset Management', 'Asylangelegenheiten', 'Auftragsmanagement', 'Auftragswesen', 'Auftragswesen', 'Auftragswesen', 'Auftragswesen', 'Auftragswesen', 'Auslandszahlungsverkehr', 'Avalkredit', 'Bankshops', 'Bankshops', 'Bankvertrieb', 'Bau', 'Bausparkassen', 'Beitragswesen', 'Benchmarking', 'Beratungskunden', 'Berechnung', 'Berechnung', 'Berechnung', 'Bergämter', 'Berichtswesen / Statistik', 'Beschaffung', 'Beschaffung', 'Beschaffung (Einkauf)', 'Beschwerdemanagement', 'Beseitigung', 'Bestand', 'Bestand', 'Bestand', 'Bestand', 'Bestand', 'Bestand', 'Bestand', 'Bestand', 'Bestand', 'Bestand', 'Bestand', 'Bestand', 'Bestandsverwaltung', 'Betriebsmittelkonstruktion', 'Betriebsmittelkonstruktion', 'Betriebswirtsch. Abrechnung', 'Bezügestellen / Kassen', 'Bibliotheken/Archive', 'Bibliotheken/Archive', 'Bilanzstrukturmgt.', 'Billing', 'Billing', 'Billing-Dienstleister', 'Bonität', 'Bundes-/Landeskriminalamt', 'Business Planung', 'Business Planung', 'Börseninformationen', 'CADCAM', 'CRM', 'Call Center', 'Call Center', 'Call Center', 'Call Center', 'Cards & Payment', 'Case-/DMP',
              'Cash-Management', 'Category Management', 'Compliance', 'Contentmanagement', 'Controlling', 'Custody', 'Customer Care', 'Customer Service Center', 'DRGs', 'Database Marketing', 'Database Marketing', 'Datendienste-Festnetz', 'Datensicherheit', 'Debitorenmanagement', 'Debitorenmanagement', 'Deckungsbeitragsrechnung', 'Depotbuchhaltung', 'Depotinformation', 'Derivate', 'Design', 'Deutsche Bundesbank', 'Deutsches Patent- und Markenamt', 'Dienstausweis', 'Direktbanken', 'Direktbanking', 'Discount Brokerage', 'Discount Brokerage', 'Distribution', 'Distribution', 'Distribution', 'Distribution', 'Dokumentärer ZV', 'Doppik', 'Doppik', 'Doppik', 'Doppik', 'Durchleitungsmgt.', 'E- und M-Services', 'E-Commerce', 'E-Commerce', 'ECR', 'EDI', 'EDI', 'ERP', 'Echtzeitprogrammierung', 'Effektive Wertpapierverwaltung', 'Eichwesen', 'Einkauf', 'Einkauf', 'Einkauf', 'Einkauf', 'Einkauf', 'Einkauf', 'Einkauf', 'Einsatzleitsysteme', 'Einzelfertigung', 'Einzelgeschäftskalkulation', 'Electronic warfare', 'Elektro', 'Endgeräteportfolio', 'Energiedatenmgt.', 'Engineer-to-order', 'Entsorgung', 'Entsorgung', 'Entsorgung', 'Ersatzteilmgt.', 'Ersatzteilmgt.', 'Erträge', 'Fachbereiche', 'Fachbereiche', 'Factory Automation', 'Fakturierung / Abrechnungsprozesse', 'Fertigungsplanung', 'Fertigungsplanung', 'Fertigungsplanung', 'Fest- und Termingelder', 'Filialorganisation', 'Filialsteuerung', 'Film', 'Finanzbuchhaltung', 'Finanzsanktionen', 'Finanzschulen', 'Finanzämter', 'Firmenkunden', 'Fondsbuchhaltung', 'Fondsinformation und Disposition', 'Forderungsmanagement', 'Forschung und Entwicklung', 'Forschung und Entwicklung', 'Forschung und Entwicklung', 'Forschungseinrichtungen', 'Forschungseinrichtungen', 'Forstdirektionen/-ämter', 'Fraudmanagement', 'Fuhrpark- / Wegeoptimierung', 'Förderwesen', 'Förderwesen', 'Gateways', 'Gattungsdaten', 'Geld- und Devisenabwicklung', 'Geldwäschebekämpfung', 'Gemeinkostencontrolling', 'Genehmigungsverfahren', 'Genehmigungsverfahren', 'Genossenschaftsbanken', 'Gerichte', 'Geschäftserfüllung', 'Geschäftsmodelle', 'Gewerbeaufsicht', 'Giro / Kontokorrent', 'Girozentralen', 'Grenzkontrolle', 'Gummi-/Kunststoffwaren', 'Güterverkehr', 'Handelsaktivitäten', 'Handelskalkulation', 'Handelskalkulation', 'Hauptversammlungen', 'Haushalt/Kasse', 'Haushalt/Kasse', 'Haushalt/Kasse', 'Haushaltswesen', 'High-Tech', 'Hochbau', 'Homebanking', 'ISP', 'Immobilienmanagement', 'Inkasso/Exkasso', 'Inkasso/Exkasso', 'Inkasso/Exkasso', 'Inkasso/Exkasso', 'Inkasso/Exkasso', 'Inkasso/Exkasso', 'Inkasso/Exkasso', 'Inkasso/Exkasso', 'Inkasso/Exkasso', 'Inkasso/Exkasso', 'Inkasso/Exkasso', 'Inkasso/Exkasso', 'Inkasso/Exkasso', 'Inlandszahlungsverkehr (inkl. EU)', 'Innerer Dienst', 'Innerer Dienst', 'Innerer Dienst', 'Instandhaltung', 'Instandhaltung', 'Instandhaltung (Facility Mgt.)', 'Internat. Gesamtrisikosteuerung', 'Internat. Roll-Out', 'Internationale Rechnungslegung', 'Internet-Billing', 'Internetbanking', 'Inventur', 'Investitionsbanken', 'Investment Guidelines', 'Just-in-time Delivery', 'Justizvollzug', 'KFZ Zulassungswesen', 'Kapazitätsplanung', 'Kapitalanlagegesellschaften', 'Kapitalveränderungen', 'Kartengestützte Bezahlverfahren', 'Kassensysteme', 'Klinische Pfade', 'Konfigurationsmanagement', 'Konstruktion', 'Konstruktion', 'Konstruktion', 'Konzernrechnungslegung', 'Konzernstrategie', 'Kosten-/Effizienzmgt.', 'Krankenhausinformationssystem', 'Krankenversicherungskarte', 'Kreditbewertung', 'Kreditverbriefung', 'Kriminalpolizei', 'Kundenanforderung', 'Kundenanschluss', 'Kundenkalkulation', 'Ladenbau', 'Lager', 'Lager', 'Lager', 'Lager', 'Lagerabwicklung', 'Lagersteuerung', 'Land-/Forstschulen', 'Landwirtschaftskammern', 'Leistung', 'Leistung', 'Leistungsbearbeitung', 'Leistungsbearbeitung', 'Leistungsbearbeitung', 'Leistungsbearbeitung', 'Leistungsbearbeitung', 'Leistungsbearbeitung', 'Leistungsbearbeitung', 'Leistungsbearbeitung', 'Leistungsbearbeitung', 'Leistungsbearbeitung', 'Leistungsbearbeitung', 'Leistungsbearbeitung', 'Leistungsbearbeitung', 'Leistungswesen', 'Letter Center', 'Liegenschaftsverwaltung', 'Liegenschaftsverwaltung', 'Limitmanagement', 'Liquiditätsmgt.', 'Logistik', 'Logistiksysteme', 'M-Applications', 'MIS / EIS', 'MIS / FIS', 'MaH Mindestanf. an Handelsg.', 'MaK Mindestanf.a.d.Kreditgesch.', 'Markenstrategien', 'Market Risk', 'Marketing', 'Marketing', 'Marketing', 'Marketing', 'Marketing', 'Marketing', 'Marketing', 'Markt-/Vertriebssteuerung', 'Marktanalysen', 'Maschinenbau', 'Materialmanagement', 'Materialprüfung', 'Materialwirtschaft', 'Materialwirtschaft', 'Materialwirtschaft', 'Materialwirtschaft', 'Materialwirtschaft', 'Materialwirtschaft', 'Materialwirtschaft', 'Medizinisches Controlling', 'Meldewesen', 'Meldewesen', 'Mergers & Acquisitions', 'Metall', 'Methodenplanung', 'Methodenplanung', 'Methodenplanung', 'Mitglied / Versicherter', 'Mitgliederverwaltung', 'Mitgliederverwaltung', 'Mitgliederverwaltung', 'Mitgliedschaft / Bestand', 'Mobile Datenerfassung', 'Montageplanung', 'Multimedia', 'Multimedia', 'Museen/Kultureinrichtungen', 'Museen/Kultureinrichtungen', 'Möbel', 'Mühlen', 'Nahrung/Getränke/Tabak', 'Netzdokumentation', 'Netzmgt.systeme', 'Netzplanung', 'Netzplanung', 'Netzplanung', 'Netzplanung', 'Netzplanung', 'Netzplanung', 'Netzsteuerung', 'Netzwartung', 'Netzwerk- u. Systemmanagement', 'OP', 'Oberfinanzdirektionen', 'Operational Risk', 'Order', 'Order', 'Order Management', 'Orderabwicklung', 'Org./DV', 'Org./DV', 'Organisationsmgt.', 'POS', 'POS', 'POS', 'POS', 'POS', 'POS', 'POS', 'POS', 'POS', 'POS', 'POS', 'POS-Banking', 'POS-Banking', 'Partner', 'Partner', 'Partner', 'Partner', 'Partner', 'Partner', 'Partner', 'Partner', 'Partner', 'Partner', 'Partner', 'Patientenmanagement', 'Performance Messung', 'Performance Mgt.', 'Performance Mgt.', 'Personal', 'Personal', 'Personal', 'Personalausweis', 'Personaldisposition', 'Personalentwicklung', 'Personalplanung', 'Planung', 'Portfoliocontrolling', 'Portfoliomanagement', 'Potenzialidentifikation', 'Potenzialidentifikation', 'Pricing', 'Pricing', 'Private Banking', 'Private Banking', 'Process Management - TOM', 'Procurement', 'Production Scheduling', 'Produkt', 'Produkt', 'Produkt', 'Produkt', 'Produkt', 'Produkt', 'Produkt', 'Produkt', 'Produkt', 'Produkt', 'Produkt', 'Produkt', 'Produkt', 'Produktionsplanung u. -steuerung', 'Produktionsplanung u. -steuerung', 'Produktionsplanung u. -steuerung', 'Produktionsplanung u. -steuerung', 'Produktionsplanung u. -steuerung', 'Produktkalkulation', 'Produktmgt.', 'Produktmgt.', 'Produktmgt.', 'Produktportfolio', 'Profitcenterrechnung', 'Programmplanung', 'Programmplanung', 'Programmplanung', 'Programmplanung', 'Programmplanung', 'Projektcontrolling', 'Prototypenbau', 'Prototypenbau', 'Prozesskontrolle', 'Prozesskostenrechnung', 'Prozessleittechnik', 'Prozessleittechnik', 'Prozessleittechnik', 'Prozessleittechnik', 'Prävention', 'Quality of Service', 'Qualitätssicherung', 'Qualitätssicherung', 'Qualitätssicherung', 'Qualitätssicherung', 'Qualitätssicherung', 'Raten-/Konsumentenkredit', 'Rating', 'Reaktorsicherheit', 'Realkreditinstitute', 'Rechnungswesen', 'Regaloptimierung', 'Regress', 'Regulatory Affairs', 'Regulierung Telco und Post', 'Rehabilitation', 'Rehabilitation', 'Rente', 'Research', 'Restrukturierung/Transformation', 'Retailbanken', 'Revision', 'Risikocontrolling', 'Risikomanagement', 'Risk Management', 'Rückversicherung passiv', 'Rückversicherung passiv', 'Rückversicherung passiv', 'Rückversicherung passiv', 'Rückversicherung passiv', 'Rückversicherung passiv', 'Rückversicherung passiv', 'Rückversicherung passiv', 'Rückversicherung passiv', 'Rückversicherung passiv', 'SB-Banking', 'SCADA Superv.Contr.a.Data Acq.', 'Sales and Distribution', 'Sammlung', 'Schaden/Leistung', 'Schaden/Leistung', 'Schaden/Leistung', 'Schaden/Leistung', 'Schaden/Leistung', 'Schaden/Leistung', 'Schaden/Leistung', 'Schaden/Leistung', 'Schaden/Leistung', 'Schaden/Leistung', 'Schaden/Leistung', 'Schengen-Verfahren', 'Schulaufsicht', 'Schulaufsicht', 'Schulen', 'Schulen', 'Schutzpolizei', 'Scoring', 'Scoring', 'Secure communications', 'Serienfertigung', 'Service Center', 'Service assurance', 'Service configuration', 'Serviceentwicklung u. Betrieb', 'Servicemgt.', 'Servicemgt.', 'Servicemgt.', 'Servicequalität', 'Settlement', 'Shop Floor Management', 'Sicherheit', 'Sicherheiten', 'Simulation', 'Simulation', 'Skill Mgt.', 'Sortierung', 'Sortimentsgestaltung', 'Spar', 'Sparkassen', 'Sport', 'Sprachdienste-Festnetz', 'Sprachdienste-Mobilfunk', 'Staatsanwaltschaften', 'Stammdaten', 'Standortsicherung', 'Steuern', 'Steuerung', 'Straßenbau', 'Studentenverwaltung', 'Supply Chain', 'Supply Chain', 'Supply Chain', 'Supply Chain', 'Supply Chain', 'Supply Chain', 'TMN', 'TQM', 'Tarifmodelle', 'Technologietransfer', 'Telefonbanking', 'Telematik', 'Testen', 'Testen', 'Textil', 'Tiefbau', 'Trading and Sales', 'Transport', 'Transport', 'Transport', 'Transport', 'Transport', 'Transport', 'Transportmgt.', 'Travel Mgt.', 'Treasury', 'Umwelt/Gefahrstoffe', 'Umwelt/Gefahrstoffe', 'Umwelt/Gefahrstoffe', 'Umwelt/Gefahrstoffe', 'Underwriting', 'VAS-Mobilfunk', 'Variantenplanung', 'Veranstaltungsmgt.', 'Verbraucherschutz', 'Verbraucherschutz', 'Verfassungsschutz', 'Vergütung', 'Vergütung', 'Vergütung', 'Vergütung', 'Vergütung', 'Vergütung', 'Vergütung', 'Vergütung', 'Vergütung', 'Vergütung', 'Verkauf', 'Vermarktung', 'Vermessung', 'Versand', 'Versand', 'Versand', 'Versand', 'Versandmanagement', 'Versicherungstechnik', 'Versuchsbau', 'Versuchsbau', 'Vertragsmgt.', 'Vertrieb', 'Vertrieb', 'Vertrieb', 'Vertrieb', 'Vertrieb', 'Vertrieb', 'Vertrieb', 'Vertrieb', 'Vertrieb', 'Vertrieb', 'Vertrieb', 'Vertrieb', 'Vertrieb', 'Vertrieb/Vertriebssysteme', 'Vertrieb/Vertriebssysteme', 'Vertrieb/Vertriebssysteme', 'Vertrieb/Vertriebssysteme', 'Vertrieb/Vertriebssysteme', 'Vertriebswege', 'Vertriebswege', 'Vertriebswege', 'Vertriebswege', 'Vertriebswege', 'Vertriebswege', 'Vertriebswege', 'Vertriebswege', 'Vertriebswege', 'Vertriebswege', 
              'Vertriebswege', 'Vertriebswege', 'Vertriebswegemgt.', 'Vertriebswegemgt.', 'Verwaltung', 'Verwertung', 'Vollmachten', 'Vollmachten', 'Vollmachten', 'Vollmachten', 'Vollmachten', 'Vollmachten', 'Vollmachten', 'Vollmachten', 'Vollmachten', 'Vollmachten', 'Vollmachten', 'Warenkorbanalyse', 'Warenwirtschaft', 'Warenwirtschaft', 'Wartung', 'Wasser-/Schifffahrtsverwaltung', 'Wasserwirtschaftsbehörden', 'Wasserwirtschaftsbehörden', 'Web access infrastructure', 'Wechsel', 'Werbung / Aktion', 'Werkstätten', 'Wertorientierte Steuerung', 'Wertpapier-Clearing', 'Wertpapierabrechnung', 'Wertpapierabwicklung', 'Wertpapiererträge', 'Wertpapierleihe', 'Wertpapierverwaltung', 'Wettbewerbsanalyse', 'Wettbewerbsanalyse', 'Wettbewerbsmanagement', 'Wirtschaftl. Zus.arbeit / Entw.', 'Wohnungsverwaltung', 'Zahlungsverkehrs-Clearing', 'Zeitwirtschaft', 'Zentralbanken', 'Zentrales Fahrzeugregister', 'Zielmarkt/-gruppenmgt.', 'Zielmarkt/-gruppenmgt.', 'Zivilschutz', 'Zählerfernauslesung', 'Ämter/Anstalten Landw.', 'Ämter/Anstalten/Fachbehörden', 'Ämter/Anstalten/Fachbehörden', 'Ämter/Anstalten/Institute', 'Ämter/Anstalten/Institute', 'Post Merger Integration', 'Integrationsstrategie', 'Governance', 'Programm Management', 'Change Management & Personal', 'Betrieb', 'Finanzen', 'IT', 'Produkte', 'Marketing & Vertrieb', 'Business Continuity Management (BCM)', 'Ethical Hacking', 'Penetrationstest', 'Telefonanlagen', 'Ericsson MD110', 'Onebox (Voiceserver)', 'Tivoli Storage Manager Server Administration', 'Tivoli Storage Manager Client Administration', 'Tivoli Storage Manager for Databases', 'Complex Event Processing (CEP)', 'Wombat', 'Schwachstellenbewertung', 'Common Vulnerabilitiy Scoring System (CVSS)', 'OWASP - Rating', 'V3.Investmentdepot', 'Finanzplanung', 'Aixigo', 'Etvice Holding', 'Xentis Profidata', 'Simcorp Dimension', 'Asset Arena (Investment Accounting)', 'Handelsaktivitäten', 'Datenmanagement', 'SAP CYT', 'CCM: Catalog & Content Management: Requisite BugsEye', 'Intrexx Xtreme', 'HP Service Center', 'Confluence', 'Kreditbearbeitungssystem', 'OSPlus', 'Green IT', 'BI Maturity Audit (biMA)', 'Business Case', 'Scrum', 'Vertriebsprozesse', 'Retailbanking', 'Private Banking', 'Business Banking', 'Private Wealth Management', 'Versicherungen', 'Mobile Vertriebe / Strukturvertriebe', 'sonstiges', 'MS Installer', 'Tivoli Backup/Recovery', 'ArcSDE', 'DB2 Host', 'DB2 UDB', 'Oracle Spatial', 'MPLS', 'SDH', 'PDH', 'Apache', 'Squid', 'DHCP', 'Rsync', 'NFS', 'CUPS', 'NTP', 'LVM', 'Biometric Matching System', 'Fingerprint Image Transmission (FIT)', 'SSH', 'IP Tables', 'ABAP Reporting', 'ABAP Transaktion', 'BC: Legacy System Migration Workbench', 'BC: Business Process Management', 'BC: Composite Application Enviroment', 'BC: Enterprise Serviceoriented  Architecture', 'BC: Visual Composer', 'BC: Adobe Document Services', 'BC: Guided Procedures', 'BSP-Entwicklung', 'BTCI Entwicklung', 'Datenbankadministration', 'DDIC', 'Formulare/Sapscript', 'HTML/XML', 'JAVA-Entwicklung', 'Migrationen Releasewechsel', 'Monitoring/Jobsteuerung', 'SAP Archivierung', 'SAP Basis Betrieb SAP CRM System', 'SAP BSP', 'SAP Forms/Smart Forms', 'SAP PI/XI', 'SAP Scripting', 'SAP Netweaver-Portal', 'SAP WebDynpro (Java und ABAP)', 'Systemadministration', 'Architekt SAP IS-U Integration', 'Anbindung SAP IS-U RTP - SAP EDM', 'Anbindung SAP IS-U - Non-SAP', 'SAP EDM - Energiedatenmanagement', 'FI/CA - Vertragskontokorrent', 'IS-U - Abrechnung/Fakturierung', 'IS-U - Datenmodell/-strukturen', 'IS-U - Formularwesen', 'IS-U - Geräteverwaltung', 'IS-U - Kaufmännische Stammdaten', 'IS-U - Kundenservice (CIC)', 'IS-U - IDE', 'IS-U - IDEX (GE und GG)', 'IS-U - Regionalstruktur', 'IS-U - RTP', 'IS-U - Tarifierung', 'IS-U - Technische Stammdaten', 'IS-U - Terminsteuerung', 'SAP Migrationsworkbench LSMW', 'SAP SLO-Tools', 'Umstellung Datenmodell SAP-Standard-Methoden', 'Umsetzung Datentransfer/Migration', 'Vorgehen Datentransfer/Migration', 'MM: Logistische Rechnungsprüfung', 'SAP CRM Analytics', 'SAP CRM Basic Customizing', 'SAP CRM E-Mail Response Mgmt (ERMS)', 'SAP CRM Groupware Integration', 'SAP CRM ICWebClient', 'SAP CRM Marketing', 'SAP CRM Middleware (IS-U,ERP,BI)', 'SAP CRM Organisationsmodell (Mapping ERP)', 'SAP CRM Sales', 'SAP CRM Service', 'SAP CRM Tel.-integr.(CTI, ACD,...)', 'SAP CRM Release Upgrade', 'SAP CRM Web Services/Web Services Wizard', 'SAP CRM WebUI Development', 'SRM: Supplier Relationship Management', 'SRM: MDM-Katalog', 'SRM: Catalog&Content Management', 'SRM: Lieferantenanbindung', 'SRM: Strategischer Einkauf', 'SRM: Operativer Einkauf', 'CO-OM-CCA (KSR)', 'CO-PC-OBJ (KTR)', 'EC-BP (Business Plan/Budgetierung)', 'FI New GL', 'FT-TRV Travelmanagement', 'Investitionsmanagement', 'TR-CM Cash Management', 'TR-TM-DE (Derivate)', 'TR-TM-FX (Devisen)', 'TR-TM-SE (Wertpapiere)', 'TR-MRM Risikomanagement', 'xRPM, cProjects', 'SD-BIL (Fakturierung)', 'SAP BI Backend', 'Datenmodellierung', 'SAP Business Information Warehouse (SAP BW)', 'Workbench', 'Performance Tuning', 'Prozessketten', 'Berechtigung', 'BI-Accelerator', 'SAP BI CPM', 'SAP - CPM (OutlookSoft)', 'SAP BI Content', 'Banking Bank Analyzer Reporting CML', 'Banking Loans', 'SAP BI Planung (BI-IP)', 'Gestaltungs Planungsprozeß', 'Konfiguration des Planungsprozesses', 'Planungslayout (WAD)', 'Planungslayout (Excel)', 'Entwicklung komplexer Planungsfunktionen', 'Status und Tracking', 'SAP BI Frontend', 'Web Application Designer', 'Business Explorer', 'Information Broadcasting', 'Report Designer', 'Portalintegration', 'Visual Composer', 'SAP BI Entwicklung', 'ABAP/Objects BI', 'Java', 'HTML', 'Visual Basic', 'SAP BI Anbindung', 'Human Resources', 'Finance', 'Logistik', 'Banking', 'Utilities', 'Real Estate', 'Health Care', 'e-Point', 'EVI (Anwendung, Datenmodell, Integrationsfähigkeit)', 'Marketing Manager Update', 'SAS Campaign Studio', 'SAS Marketing Optimization', 'Siebel 7.7 / 7.8', 'Siebel 8.X', 'Siebel Eim', 'Siebel Sales/Marketing', 'Siebel Call Center', 'CRM fachlich', 'Kampagnenmanagement', 'Kundenservice-Center', 'Beschwerdemanagement', 'Kundenbindung/Loyalty', 'Vertriebsunterstützung', 'PrintMachine', 'Bloomberg POMS', 'Calypso', 'IBM Lotus Domino Enterprise Server', 'IBM WebSphere Portal Server', 'IBM WebSphere Message Broker', 'Deutsche Post MailOptimizer', 'PrintMachine', 'Cards&Payments Standards', 'Karten- / Chip-Technologie (EMV, SECCOS)', 'ISO 8583', 'Digital Payments allgemein', 'Card Payment Schemes Debit', 'Card Payment Schemes allgemein', 'Card Payment Schemes Kredit', 'Ascent Capture', 'Ascent Xtrata Pro', 'Interconnect', 'Info Vista', 'Metrica/NPR', 'Standardsoftware Utilities', 'SOPTIM VIM/PMS', 'Robotron eSales', 'Seeburger', 'Web Mapping', 'ArcIMS', 'Deegree', 'UMN - Mapserver', 'Basel II', 'Cognos ReportNet', 'IBM Cognos 8 BI Report Studio', 'IBM Cognos 8 BI Analysis Studio', 'IBM Cognos 8 BI Query Studio', 'IBM Cognos 8 BI Event Studio', 'IBM Cognos 8 BI Framework Manager', 'IBM Cognos 8 BI Transformer', 'Cognos Transformer', 'IBM Cognos 8 BI Map Manager', 'IBM Cognos 8 BI Administration', 'IBM Cognos 8 BI SDK', 'Hyperion Essbase', 'Hyperion Reporting', 'Oracle BI Enterprise Edition', 'Oracle BI Query & Analysis', 'SAS Analytic Intelligence', 'SAS BI (Frontend, OLAP, Information Maps)', 'SAS Data Integration Studio/ETL', 'SAS unter Unix', 'SAS unter ZOS', 'Data Mining', 'Cluster Analyse', 'Klassifikation', 'Zeitreihen-Analyse', 'Scoring Methode', 'Spatial Data Mining', 'Hibernate', 'DES / OM (Generation & Output)', 'Modus Suite', 'Papyrus', 'Assentis', 'ARIS Toolset Datenmodellierung', 'CVS / PVCS', 'Visual Source Safe', 'Jbuilder', 'Together', 'SER (Doxis)', 'TRIBUTUM', 'Dynamische Schnittstelle (SI)', 'Oracle RDBMS', 'Oracle Warehouse Builder', 'Oracle Reports', 'Oracle Express', 'Oracle OLAP', 'Ab Initio', 'Ab Initio EME-Administration', 'Ab Initio Installation und Konfiguration', 'IBM DataStage Server Edition', 'MS SQL Server Integration Service', 'Informatica Power Center', 'systemfabrik', 'Brio', 'BusinessObjects Desktop Intelligence', 'Business Objects Enterprise Universe Designer', 'BusinessObjects Backend, Server, Administration, Security', 'BusinessObjects Crystal Reports', 'BusinessObjects Crystal Reports Server', 'BusinessObjects Xcelsius', 'BusinessObjects Data Integrator: Batch Jobs', 'BusinessObjects Data Integrator: Web Services', 'BusinessObjects Data Integrator: Data Quality (IQ8, First Logic)', 'BusinessObjects Data Integrator: Metadata Manager', 'BusinessObjects Planning Produkte', 'MIS Alea', 'Applix/TM1', 'MicroStrategy', 'Oracle Dashboards', 'Oracle Reporting & Publishing', 'AJAX', 'XBAP - XAML Browser Application', 'Glassfish 2', 'Sun Application Server', 'Tomcat', 'Java Swing', 'Windows Presentation Foundation', 'Intrexx Xtreme', 'Python', 'Prozessmanagement', 'Konzeption Prozessabbildung', 'Six Sigma / Lean', 'Kundenkommunikation', 'Vorstandskommunikation', 'Anforderungsaufnahme/Analyse', 'Projektleitung', 'Teilprojektleitung', 'Offshore-Projektmanagement', 'Nearshore-Projektmanagement', 'Offshore Projektplanung / -Steuerung', 'Nearshore Projektplanung / -Steuerung', 'Räumliche Analyseverfahren', 'Point Pattern Analyse', 'Spatial Data Mining', 'NUNIT', 'HP Quality Center (QC) / Application Lifecycle Management (ALM)', 'Borland Silktest', 'Testmanagement', 'Software Lifecycle Management', 'Team System', 'Evaluierung', 'Team Foundation Server', 'Einführung', 'Migration / Betrieb', 'Anpassung / Customization', 'Software Entwicklungsprozesse', 'Analyse', 'Beratung', 'Einführung', 'Anpassung', 'Reporting', 'SLM Unterstützende Prozesse', 'Requirements Engineering', 'Configuration Management', 'Quality Management', 'Build Management', 'Deployment Management', 'Release Management', 'Software Configuration Management', 'Testtools', 'NUnit', 'MS Test', 'MS Build', 'Favorit 4.1', 'DOMEA© 4.0 / 4.5', 'Fabasoft eGov-Suite 6.0', 'Favorit 4.1', 'Standards', 'DMS', 'Domea 2.1', 'MoReq 2', 'ARIS', 'ADONIS', 'VISIO', 'Oracle Data Integrator', 
              'Oracle Streams', 'Oracle Change Data Capture', 'BUS Architektur', 'SAP BusinessObjects Enterprise XI', 'SAP BusinessObjects Data Services', 'Jasper ETL', 'ELT-Ansatz', 'Near Real Time Bewirtschaftung', 'Informationsbedarfsanalyse', 'BI Governance (biGov) ', 'Kennzahlen-Dimensionen-Modell', 'Profiling', 'DFPS', 'IT-Service Management', 'Helpdesk', 'Service Level Agreements', 'Mantis', 'Serena TeamTrack', 'ClearQuest', 'ChangeSynergy', 'JMeter', 'DBUNIT', 'QFS QF-Test', 'Borland Silk Central', 'Testmanagementtools', 'Lasttest', 'Performancetest', 'Desaster- und Recoverytest', 'Regressionstest', 'Sicherheitstest', 'Testdatenmanagement', 'Testadministration', 'Statische Testverfahren', 'Capture & Replay', 'Testdurchführung', 'Teststrategie', 'Quality Gates', 'ITIL', 'Metadatenmanagement', 'Schulungsmanagement', 'Sarbanes Oxley Compliance (SOX)', 'QM-Tool Training', 'Schulungsentwicklung/ und -konzeption', 'Business Integration (Tools + Methoden)', 'Unternehmensplanung', 'Business Document Automation Outputmanagement', 'Cards & Payments', 'Allgemein', 'Produktentwicklung / Kartenstrategien', 'Marketing, Vertrieb', 'Issuing / Issuing Processing', 'Acquiring / Acquiring Processing', 'Transaction Processing', 'Kartenmanagement', 'Customer Care / Dispute Handling', 'Finanzierung und Risikomanagement', 'Routing / Switching / Autorisierung', 'Fraud Combating', 'GAA-, POS-Betrieb und zugehörige Netze', 'Clearing and Settlement', 'Leasing', 'Risikomanagement', 'Basel II', 'MaRisk', 'Fondsabwicklung', 'Fondssparpläne', 'WP-Provisionen', 'Neuemissionen', 'WP-Abstimmung', 'Fiskalische Informationen', 'Auslandszahlungsverkehr (Non SEPA)', 'IZV (Nationaler ZV)', 'Clearing and Settlement', 'SEPA  (Europäischer Zahlungsverkehr)', 'SWIFT Messaging', 'Customer Care', 'Gesundheitspolitik', 'Vertragsmanagement', 'Disease Staging', 'Provisionsberechnung', 'Fault Management', 'Performance Management', 'Luftfracht', 'Seefracht', 'Kurier-Express-Paket KEP', 'Straßentransport', 'Speditionswesen', 'Stromversorger Unternehmenssteuerung', 'Stromversorger Privatkunden Vertrieb', 'Stromversorger Großkunden Vertrieb', 'Gasversorger Unternehmenssteuerung', 'Gasversorger Privatkunden Vertrieb', 'Gasversorger Großkunden Vertrieb', 'Vertriebsstrategie', 'Vertriebsstrategie', 'Vertriebsstrategie', 'Vertriebsstrategie', 'Unternehmensführung Konzern', 'Unternehmensführung Konzern', 'Unternehmensführung Regionalversorger', 'Unternehmensführung Regionalversorger', 'Unternehmensführung Stadtwerk', 'Unternehmensführung Stadtwerk', 'Geschäftsfeldsteuerung', 'Geschäftsfeldsteuerung', 'Ergebnisrechnung IFRS', 'Ergebnisrechnung IFRS', 'Ergebnisrechnung HGB', 'Ergebnisrechnung HGB', 'Strategisches Berichtswesen Konzern', 'Strategisches Berichtswesen Konzern', 'Strategisches Berichtswesen Regionalversorger', 'Strategisches Berichtswesen Regionalversorger', 'Strategisches Berichtswesen Stadtwerk', 'Strategisches Berichtswesen Stadtwerk', 'Geschäftsstrategie Konzern', 'Geschäftsstrategie Konzern', 'Geschäftsstrategie Regionalversorger', 'Geschäftsstrategie Regionalversorger', 'Geschäftsstrategie Stadtwerk', 'Geschäftsstrategie Stadtwerk', 'Rechnungswesen IFRS', 'Rechnungswesen IFRS', 'Rechnungswesen HGB', 'Rechnungswesen HGB', 'Geschäftsfeldcontrolling', 'Geschäftsfeldcontrolling', 'SLA Vertrieb/Netz/Shared Service', 'SLA Vertrieb Netz', 'SLA Vertrieb Kundenservice', 'SLA Vertrieb IT', 'Risikocontrolling', 'Risikocontrolling', 'Portfoliomanagement', 'Portfoliomanagment', 'Produktportfolio', 'Produktportfolio', 'Produktportfolio', 'Produktportfolio', 'Strategisches Marketing', 'Strategisches Marketing', 'Strategisches Marketing', 'Strategisches Marketing', 'Organisation Strom/Gas', 'Organisation Strom/Gas', 'Organisation Strom/Gas', 'Benchmarking Organisation', 'Benchmarking Prozesse', 'Benchmarking Prozesse', 'Akquiseprozess (Kampagnen)', 'Akquiseprozess (Kampagnen)', 'Akquiseprozess (Kampagnen)', 'Akquiseprozess (Kampagnen)', 'Angebotsprozess', 'Angebotsprozess', 'Angebotsprozess', 'Angebotsprozess', 'Folgevertrag', 'Folgevertrag', 'Folgevertrag', 'Folgevertrag', 'Preisanpassung', 'Preisanpassung', 'Preisanpassung', 'Preisanpassung', 'Beschaffungsstrategie', 'Beschaffungsstrategie', 'Beschaffungsstrategie', 'Beschaffungsprozesse', 'Beschaffungsprozesse', 'Beschaffungsprozesse', 'Produktentwicklung', 'Produktentwicklung', 'Produktentwicklung', 'Produktentwicklung', 'Marketing', 'Marketing', 'Marketing', 'Marketing', 'Lastgangverwaltung/Profile', 'Lastgangverwaltung/Profile', 'Lastgangverwaltung/Profile', 'Lastgangverwaltung/Profile', 'Prognose', 'Prognose', 'Prognose', 'Prognose', 'Prognoseoptimierung', 'Prognoseoptimierung', 'Prognoseoptimierung', 'Prognoseoptimierung', 'Kundenservice/Service Center', 'Kundenservice/Service Center', 'Kundenservice/Service Center', 'Kundenservice/Service Center', 'Call Center', 'Call Center', 'Call Center', 'Call Center', 'Forderungsmanagement', 'Forderungsmanagement', 'Forderungsmanagement', 'Forderungsmanagement', 'Debitorenbuchhaltung', 'Debitorenbuchhaltung', 'Debitorenbuchhaltung', 'Bilanzierung', 'Bilanzierung', 'Bilanzierung', 'Bilanzierung', 'Bilanzierungsverordnungen', 'Bilanzierungsverordnungen', 'Verbuchungsprozesse', 'Verbuchungsprozesse', 'Verbuchungsprozesse', 'Hauptbuch Anlagensteuerung', 'Investitions-/Planungsprozesse', 'MABIS', 'MABIS', 'GABI', 'GABI', 'Materialverwaltung', 'Materialbeschaffung', 'Geräteverwaltung', 'Geräteverwaltung', 'Geräteverwaltung', 'Geräteverwaltung', 'Beglaubigungsprozesse/Eichwesen', 'Beglaubigungsprozesse/Eichwesen', 'Hausanschluss/Inbetriebnahme', 'Hausanschluss/Inbetriebnahme', 'Ablesung/Ablesesteuerung', 'Ablesung/Ablesesteuerung', 'Ablesung/Ablesesteuerung', 'Ablesung/Ablesesteuerung', 'Zählwerterfassung', 'Zählwerterfassung', 'Zählwerterfassung', 'Zählwerterfassung', 'Messstellenbetrieb', 'Messstellenbetrieb', 'Messstellenbetrieb', 'Messstellenbetrieb', 'Smart Metering', 'Smart Metering', 'Smart Metering', 'Smart Metering', 'Messung Strom', 'Messung Strom', 'Messung Strom', 'Messung Strom', 'Messung Gas', 'Messung Gas', 'Messung Gas', 'Messung Gas', 'Zähler Strom', 'Zähler Strom', 'Zähler Strom', 'Zähler Strom', 'Zähler Gas', 'Zähler Gas', 'Zähler Gas', 'Zähler Gas', 'Abrechnungsprozesse Gas', 'Abrechnungsprozesse Gas', 'Abrechnungsprozesse Gas', 'Abrechnungsprozesse Gas', 'Abrechnungsprozesse Strom', 'Abrechnungsprozesse Strom', 'Abrechnungsprozesse Strom', 'Abrechnungsprozesse Strom', 'Abrechnungsprozesse Kombi Strom/Gas', 'Abrechnungsprozesse Kombi Strom/Gas', 'Abrechnungsprozesse Kombi Strom/Gas', 'Abrechnungsprozesse Kombi Strom/Gas', 'Abrechnungssteuerung', 'Abrechnungssteuerung', 'Abrechnungssteuerung', 'Abrechnungssteuerung', 'Formularwesen Kundenservice', 'Formularwesen Kundenservice', 'Formularwesen Kundenservice', 'Berichtswesen KS (strategisch)', 'Berichtswesen KS (operativ)', 'Berichtswesen KS (operativ)', 'Berichtsanforderung Lieferant', 'Berichtsanforderung Bundesnetzagentur', 'Berichtsanforderung Bundesnetzagentur', 'Beschwerde-/Reklamationsmanagment', 'Beschwerde-/Reklamationsmanagment', 'Beschwerde-/Reklamationsmanagment', 'Beschwerde-/Reklamationsmanagment', 'Liefermgmt./Lieferantenwechsel', 'Liefermgmt./Lieferantenwechsel', 'Liefermgmt./Lieferantenwechsel', 'Liefermgmt./Lieferantenwechsel', 'Datenaustausch EDIFACT Strom', 'Datenaustausch EDIFACT Strom', 'Datenaustausch EDIFACT Strom', 'Datenaustausch EDIFACT Gas', 'Datenaustausch EDIFACT Gas', 'Datenaustausch EDIFACT Gas', 'Datenaustausch EDIFACT Gas', 'Eingangsrechnungsprüfung', 'Eingangsrechnungsprüfung', 'Eingangsrechnungsprüfung', 'Eingangsrechnungsprüfung', 'Eingangsrechnungsverarbeitung', 'Eingangsrechnungsverarbeitung', 'Eingangsrechnungsverarbeitung', 'Eingangsrechnungsverarbeitung', 'Bundesnetzagentur Verordnungen Strom', 'Bundesnetzagentur Verordnungen Strom', 'Bundesnetzagentur Verordnungen Strom', 'Bundesnetzagentur Verordnungen Strom', 'Bundesnetzagentur Verordnungen Gas', 'Bundesnetzagentur Verordnungen Gas', 'Bundesnetzagentur Verordnungen Gas', 'Bundesnetzagentur Verordnungen Gas', 'Prozesse Dienstleistungen bei EVU', 'Prozesse Dienstleistungen bei EVU', 'Prozesse Dienstleistungen bei EVU', 'Prozesse Dienstleistungen bei EVU', 'Berichtswesen Vertrieb (strategisch)', 'Berichtswesen Vertrieb (operativ)', 'SAS Hub&Spoke', 'IBM WebSphere Application Server Network Deployment', 'IBM WebSphere Portal Server', 'JMS', 'SAS Logscanner', 'Maven', 'Storage (SAN, NAS, DAS)', 'Confirmation', 'MaRisk VA', 'Solvency II', 'Temenos - T24', 'Ab Initio Continuous Flows (Echtzeitverarbeitung)', 'Ab Initio Meta Programming', 'Ab Initio BRE (Business Rule Environment)', 'IBM Cognos 8 BI Data Manager', 'IBM Cognos 8 BI PowerPlay Studio', 'IBM Cognos 8 BI PowerPlay Client', 'IBM Cognos 8 BI Analysis for Microsoft Excel', 'IBM Cognos 8 Planning', 'IBM Cognos TM1', 'Core Banking', 'OSPlus', 'EuroSIG', 'bank21', 'GEOS', 'Kordoba', 'Himalaya', 'EFDIS', 'agree', 'MBS', 'Avaloq - Avaloq Banking System', 'Stammdaten', 'Insurance', 'Public Sector', 'CRM', 'BusinessObjects Analyzer', 'BusinessObjects Live Office', 'BusinessObjects Dashboard Builder', 'BusinessObjects Enterprise Administration', 'Change Data Capture CDE', 'IBM IIS Metadata Workbench', 'Model-Driven Software Development', 'openArchitectureWare', 'Scrum', 'Unternehmenssteuerung', 'Linienflugverkehr', 'Web Dynpro: Entwicklung von Erfassungsmasken im WEB', 'SAP Portalintegration', 'Business Objects Planning & Consolidation (BPC)', 'IBM Rational Application Developer', 'WebSphere Integration Developer', 'WebSphere Enterprise Service Bus', 'HP Quality Center', 'Technologie & Netzarchitektur', 'Long Term Evolution (LTE)', 'IP Multimedia Subsystem (IMS)', 'Voice over LTE via Generic Access ( VoLGA)', 'Self Organising Network ( SON)', 'FemtoCell ( UMTS/LTE)', 'UMTS', 'GSM', 'MPLS', 'GPRS', 'Microsoft Dynamics CRM', 'Chordiant Marketing Director', 'Chordiant Decision Management', 'Testprozessverbesserung', 'Teradata', 'EJB 3',
              'JPA (Java Persistence API)', 'ArcGIS Desktop', 'ArcGIS Server', 'ArcView 3.x', 'MapInfo Pro', 'Quantum GIS', 'gvSIG', 'GeoMedia WebMap', 'MapXtreme', 'MapBender', 'GeoServer', 'OGC', 'WMS', 'WFS', 'CSW', 'GML', 'Simple Features', 'OpenLayers', 'PostGIS', 'Open Text Websolutions', 'Coremedia', 'Enterprise Search', 'Web Analytics', 'Java ME', 'Konsortialfinanzierung', 'Projektfinanzierung', 'Baufinanzierung', 'Hyperion Planning', 'Projektcontrolling', 'Aspekt Oriented Programming', 'Windows Applications', 'Windows Presentation Foundation', 'Automatisierung', 'Micrsoft .NET UI Automation', 'Standardsoftware Compliance', 'Norkom', 'Actimize', 'Oracle 11g', 'Oracle 11g', 'Oracle Database Vault', 'Oracle Grid Control', 'Oracle Label Security', 'Oracle Active Data Guard', 'Oracle Real Application Testing', 'Oracle Total Recall', 'Oracle Application Express (APEX)', 'Oracle OLAP', 'Oracle Datenbank', 'Openlink Endur', 'MATLAB', 'OSGi', 'Windows Server 2008', 'Enterprise Architect', 'Talend Open Studio', 'OSGi', 'Springsource dm Server', 'Zoll', 'Vorgangsverwaltung', 'MS Visual Studio Team System', 'BPO Engine', 'codeBeamer', 'Microsoft Team Foundation Server', 'Build Management', 'Release Management', 'OSPlus', 'PMS-Prozesse/Dynamische Schnittstellen', 'OSPlus-Portal', 'Vetrags DB', 'PMOplus (Finanz Informatik)', 'Scrum', 'CRM methodisch', 'Kampagnenmanagement', 'Call-, Service Center', 'Vertriebsunterstützung', 'Beschwerdemanagement', 'Compliance', 'CMS (Compliance Management System)', 'Geldwäscheprävention', 'Betrugsprävention', 'Korruptionsprävention', 'Finanzsanktionen / Embargo', 'Bekämpfung der Terrorismusfinanzierung', 'Kapitalmarkt Compliance', 'Cybercrime', 'Smaragd', 'Mantas', 'WorldCheck', 'Factiva', 'Compliance & Ethics Management', 'Konzeption und Durchführung von Risikoanalysen', 'Konzeption und Etablierung Compliance Organisation (Aufbau-/ Ablauforganisation)', 'Konzeption und Umsetzung von Kontrollsystemen und -maßnahmen', 'Konzeption und Umsetzung von Schulungssystemen und -maßnahmen', 'Konzeption und Implementierung von Monitoring- und Reportingsystemen/-maßnahmen', 'Fraud Prevention und Anti-Korruption', 'Konzeption und Durchführung von Gefährdungsanalysen', 'Konzeption und Umsetzung von Präventions- und Reaktionsmaßnahmen', 'Compliance Due Diligence', 'Compliance Organisation', 'Gefährdungsanalyse', 'Prozessanalyse', 'Risikoanalyse', 'Kongruenzanalyse', 'Konzeption u Implementierung v Richtliniensystematiken u Standards: Verhaltenskodex, Org.anweisungen', 'Internationales Recht/Völkerrecht', 'UN Sicherheitsrats Resolutionen 1267 &1333', 'FATF 40 + 9 Empfehlungen', 'Europäisches Recht', 'Umsetzung der 3 EU Geldwäscherichtlinien', 'EU Verordnungen über Finanzsanktionen 2580/2001 & 881/2002', 'Europarat Konventionen', 'Konvention von Strasbourg (1990)', 'Konvention von Warschau (2005)', 'UN Konventionen', 'UN Konvention über Unterdrückung von Terrorismus Finanzierung (1999)', 'UN Konvention gegen organisierte Kriminalität (2000)', 'UN Konvention gegen Korruption (2003)', 'QlikView Skript', 'QlikView Frontend', 'QlikView Server/Publisher', 'QlikView SAP Connector', 'BI Strategy (biSE)', 'HP Fortify 360', 'Einführung', 'Anpassung / Customization', 'Kofax Capture', 'Kofax Transformation Modules', 'Business Objects Planning & Consolidation (BPC)', 'ABAP/4', 'Web Dynpro: Entwicklung und Erfassungsmasken im WEB', 'BSP (Business Server Pages)', 'Business Objects Enterprise InfoView', 'BusinessObjects Enterprise, Server, Auditing, Publishing', 'IBO', 'IBO', 'IBO', 'HP Quality Center', 'Aufsichtsrecht', 'Versicherungsrecht', 'Actimize', 'ABACUS/DaVinci', 'Basel III', 'Polarion Requirements', 'Agiles Projektmanagement', 'ESQL', 'IBM WebSphere Message Broker Toolkit', 'Enterprise Architektur', 'alfabet planningIT', 'ADOit', 'IBM Rational System Architect', 'iteraplan', 'Mega', 'Troux', 'Hudson', 'IBM Rational Team Concert', 'SOA Service Portfolio Management', 'SOA Governance', 'Enterprise Architektur', 'EA-Strategie', 'EA-Prozesse', 'EA-Content', 'icon DOPE', 'JEE 6', 'Drools', 'Guvnor', 'Governikus WebSigner', 'Facelets', 'JBoss Seam', 'OSCI', 'JBPM', 'Corporate Performance Management', 'Bestands-/ Beitragscontrolling', 'Servicecontrolling', 'Unternehmenscontrolling', 'Vertriebscontrolling', 'PHPUnit', 'Android', 'iOS', 'Mobile Betriebssysteme', 'Android', 'iOS', 'Netzmanagement', 'EDM', 'Controlling', 'IT-Architektur', 'Marktkommunikation', 'Messdienstleistung', 'Regulierung', 'SAP CRM Produkte', 'SAP CRM Integration CRM - IS-U', 'SAP CRM Integration CRM - BI', 'SAP CRM Easy Enhancement Workbench (EEWB)', 'SAP CRM Kalkulator / PCU', 'SAP CRM Customizing (gesamt)', 'SAP CRM Konfigurator / PME', 'SAP CRM Pricing/ IPC', 'SAP CRM Billing', 'SAP CRM Financial Customer Care (fachlich)', 'SAP CRM Web UI', 'SAP CRM 7.0 Development', 'SAP eSOA', 'SAP IS-U BI', 'SAP CRM Extraktion', 'SAP Retraktion (CRM + CO)', 'SAP Retraktion (FI / FI-CA / CO / CO-PA)', 'SAP CRM Retraktion (FI / FI-CA / CO / CO-PA)', 'non-SAP (DB-/UI-Connect)', 'SAP BPEM (BW Integration)', 'SAP Business Objects', 'BI Migration -> BW 7.0', 'SAP CRM 2007 fachlich', 'SAP CRM 2007 technisch', 'SAP CRM 7.0 fachlich', 'SAP CRM 7.0 technisch', 'SAP CRM Orgmodell, Rollen und Berechtigungen', 'Kvasy', 'GAS-X', 'IS-U Customizing', 'IS-U - IDEX (GM)', 'IS-U - UIS', 'SAP Migrationsworkbench EMIGALL', 'Microsoft Application Virtualization', 'RT.easy Framework (technisch)', 'Transfer Agency', 'Fondsverwaltung (Back Office)', 'Transfer Agency', 'Pareto', 'Kaizen', 'KVP', 'Design of Experiments (DOE)', 'TRIZ', 'Minitab 15', 'DMAIC (bestehende Prozesse)', 'Green Belt Training', 'Mitarbeiterschulungen Yellow Belt', 'Lean Management', 'Lean Six Sigma', 'Lean Six Sigma', 'Prozessmanagement', 'FinTS (Financial Transaction Services)', 'Informationssicherheit - organisatorisch / prozessual', 'Informationssicherheitsorganisation', 'Informationssicherheitsprozesse', 'IAM - Management Berechtigungsprozesse', 'Computer Emergency Response Team (CERT)', 'Disaster Recovery', 'HR Screening', 'Information Security Management System (ISMS)', 'IT-Compliance Management', 'Patch Management', 'Portable Media Management', 'IAM - Rezertifizierung (Identitäten, Berechtigungen, SoD-Risiken)', 'IAM - Rollenmanagement (Berechtigungsstruktur)', 'Security Awareness und Ausbildung', 'Security Incident Management', 'Sicherheitsleitlinienerstellung', 'IAM - physisches Zutrittsmanagement', 'Informationssicherheitsmethoden', 'Bedrohungsanalyse', 'Business Impact Analyse', 'Code Review', 'Datenklassifizierung', 'Design for Security', 'Forensik', 'IT-Audit', 'Return of Security Invest Berechnung', 'Risikoanalyse', 'Sicherheitskonzepte', 'STRIDE', 'Vulnerability Scanning', 'Informationssicherheit - technologisch', 'Sicherheitstechnologien', 'Abhörsicherheit', 'Archivierung', 'Authentifizierung - digitale Zertifikate', 'Authentifizierung - neuer Personalausweis (nPA)', 'Authentifizierung - SmartCards', 'Authentifizierung - token-basiert', 'Data Loss Prevention', 'Datenbanksicherheit', 'Digitale Signatur', 'E-Mail-Sicherheit - Anti-Spam', 'E-Mail-Sicherheit - vertrauliche Übermittlung', 'IAM - Federation', 'IAM - Identity Theft', 'IAM - Privileged Identity and Access Management', 'Mobil - Plattform - Android', 'Mobil - Plattform - Blackberry', 'Mobil - Plattform - iOS', 'Mobil - Plattform - Windows Mobile', 'Mobil - Devices - Laptops', 'Mobil - Devices - Radio Frequency Identification (RFID)', 'Mobil - Devices - Smartphones', 'Mobil - Devices - Tablet PCs', 'Mobil - Bring Your Own Device (BYOD)', 'Mobil - Mobile Device Management (MDM)', 'Netzwerk - Application Level Gateway', 'Netzwerk - Bluetooth', 'Netzwerk - Content Filtering', 'Netzwerk - GSM Sicherheit', 'Netzwerk - Network Access Control', 'Netzwerk - Proxy', 'Netzwerk - Secure Remote Access', 'Netzwerk - Wireless Local Area Network (WLAN)', 'Perimeterschutz', 'Rechenzentrumssicherheit', 'SAP-Sicherheit', 'Steganografie', 'Systemhärtung - Windows', 'Systemhärtung - Unix', 'Systemhärtung - Linux', 'Systemsicherheit - Backup & Recovery', 'Systemsicherheit - Client', 'Systemsicherheit - Cloud Computing', 'Systemsicherheit - Hochverfügbarkeit', 'Systemsicherheit - RACF', 'Systemsicherheit - Security Information and Event Management (SIEM)', 'Systemsicherheit - TK-Anlagen', 'Verschlüsselung - Elliptische Kurven', 'Verschlüsselung - RSA', 'Verschlüsselung - symmetrisch', 'Sicherheitsprotokolle und -datenformate', '802.1X', 'IPsec', 'OVAL', 'PAP/CHAP/EAP', 'PKCS', 'Radius', 'S/MIME', 'SAML', 'SFTP', 'TKIP', 'WEP', 'WPA/PSK', 'XCCDF', 'Hersteller von Sicherheitsprodukten', 'Checkpoint', 'Quest', 'Sophos', 'Symantec', 'Tenable', 'Informationssicherheit - rechtlich / regulatorisch', 'Gesetze', 'DE: BDSG', 'DE: BilMoG', 'DE: GdPDU', 'DE: IuKD', 'DE: KonTraG', 'DE: KWG', 'DE: SigG', 'DE: StGB §203', 'DE: TDG', 'EU: Directive 2006/43/EG', 'EU: Directive 95/46/EG\xa0', 'EU: Model Clauses', 'US: FDA', 'US: HIPAA', 'US: Sarbanes-Oxley Act (SOX)', 'Andere Vorschriften', 'Aufbewahrungspflichten', 'PCI DSS (Payment Card Ind. Data Security Standard)', 'UP-Bund', 'Informationssicherheit - Standards', 'BSI 100-x', 'ISO 22301', 'ISO 13335-3', 'ISO 15408 (Common Criteria)', 'ISO 2700x', 'Führungsfähigkeit', 'Führungsverfahren', 'Führungsunterstützung', 'Personalmanagement', 'Sanitätsdienstliche Versorgung', 'Logistische Unterstützung', 'Mobilität', 'Standard-Anwendungs-Software-Produkt-Familien (SASPF)', 'Windows7', 'CSS', 'JavaScript', 'Output Management System ', 'ICON DOPE', 'Qualifikationen Testing', 'Standardsoftware Real Estate', 'GES', 'WohnData', 'Blue Eagle', 'Verschlüsselung - asymmetrisch', 'EMIR', 'Dodd-Frank-Act', 'MiFiD', 'Products FX/MM', 'Products IRD', 'Products CRD', 'Products Securities', 'Products Securities Financing', 'Products Equities', 'Products Commodities', 'Urdu', 'Selenium', 'Inubit', 'IBM Rational Synergy', 'Sonar', 'Testing - Werkzeuge', 'Requirement Engineering', 
              'HP Quality Center (QC) / Application Lifecycle Management (ALM)', 'Polarion REQUIREMENTS', 'Testfallerstellung', 'Smartesting Certifylt', 'Testmanagement', 'IBM Rational', 'Polarion QA/ALM', 'SAP Solution Manager', 'SQS Test Professional Suite', 'Tricentis TOSCA', 'Nicht-funktionale Tests', 'HP Loadrunner', 'Testautomatisierung', 'HP Quick Test Professional (QTP) / Unified Functional Testing (UFT)', 'Odin Axe', 'Qualisystems Testshell', 'SAP eCATT', 'Tricentis TOSCA', 'Testdatenmanagement', 'gfb Q-Up', 'SAP TDMS', 'Testing - Prozesse', 'Teststufen', 'Systemtest', 'Abnahmetest', 'Integrationstest', 'Testverfahren', 'Smoketest', 'User Acceptance Test', 'Testorganisation', 'Testen mit verteilten Teams', 'Near-/Offshore Testing', 'Testen in agilen Umgebungen', 'Testing - Methoden', 'Testassessment/-optimierung', 'PIQUET', 'TPI Next', 'TMMi', 'Testdesign', 'Risk Based Testing', 'Testautomation', 'Key Word Driven', 'Testdatenmanagement', 'Testdatenanonymisierung', 'Testdatengenerierung', 'Testdatenversionierung', 'Testdatenarchivierung', 'Testumgebungsmanagement', 'Testdaten Lade/Entlade-Routinen', 'Testinfrastruktur-Management', 'Testautomationsframework', 'Testing - Ausrichtung', 'Funktionale Tests', 'Test@SAP', 'Test@BI', 'Test@SEPA', 'Test@Banking', 'Test@Insurance', 'Test@Public Services', 'Test@Telko', 'Test@Utilities', 'Test@MRS', 'Nicht-funktionale Tests', 'Stresstest', 'Entwicklertest', 'SAP HANA', 'Installation & Administration', 'Implementierung & Modellierung', 'SAP Business Objects Mobile ', 'SAP Predictive Analysis', 'Capital Markets - Produkte', 'Optionen', 'Futures', 'Börsengehandelte Derivate', 'OTC Derivatives ', 'Optionsscheine', 'Digital Options', 'Barrier Options', 'Asian Options', 'Compound Options', 'FX Spots/Forwards', 'FX Swaps', 'FX Options', 'FX Non Deliverable Forwards', 'Tages- und Termingelder', 'Certificates of Deposit', 'Commercial Papers', 'Forward Rate Agreements', 'Overnight Index Swaps', 'Geldmarktfutures', 'Bonds', 'Asset Backed Securities', 'Mortgage Backed Securities', 'Structured Covered Bonds', 'Pfandbriefe', 'Schuldscheindarlehen', 'Repos', 'Securities Lending', 'Forward Rate Agreements', 'Interest Rate Swaps', 'Swaptions', 'Cross Currency Swaps', 'Cap, Floor, Collars', 'Floating Rate Notes', 'EONIA Swaps', 'Constant Maturity Swaps', 'Credit Default Swaps', 'Credit Linked Notes', 'Total Return Swaps', 'Collateralised Debt Obligations', 'nth to Default Swaps', 'Synthetic CDOs', 'Aktien', 'Aktien Optionen', 'Aktien Futures', 'Aktien Index Derivate', 'Equity Swaps', 'Power', 'Oil', 'Freight', 'Agriculture', 'Carbon Emmissions', 'Aktienanleihen', 'Index-Linked Bonds', 'Dual Currency Bonds', 'Discountzertifikate', 'Strukturierte Anleihen', 'Kapitalschutz-Zertifikate', 'Express-Zertifikate', 'Bonus-Zertifikate', 'Index-/Partizipations-Zertifikate', 'Knock-Out Produkte', 'Inflation linked Bonds', 'Contracts for Difference', 'Capital Markets - Prozesse', 'Marktdaten', 'Gattungsdaten', 'Zinskurven', 'Kontrahentendaten', 'Standard Settlement Instructions', 'Feiertagskalender', 'Indexberechnungen', 'Emissionsprozesse', 'Verbriefung', 'Strukturierung und Bewertung', 'Deal Capture', 'Internal Deals', 'Algo Trading', 'High Frequency Trading', 'Liquidity Management', 'Asset Liability Management', 'Collateral Management', 'ISDA', 'Credit Support Annex (CSA)', 'Liquitity Risk Management', 'Hedge Accounting', 'Funding', 'Ordertypen', 'Ordermanagement', 'Positionsführung', 'P&L Berechnungen', 'Marktgerechtigkeit', 'Master Agreements', 'Limits & Exposure Management', 'Collateral Management', 'Reconciliation', 'Lifecycle Management verbriefter Derivate', 'Trade Validation & Market Operations', 'Corporate Actions', 'CLS', 'Confirmations', 'Trade Repositories', 'Bestandspflege (Fixings & Maturities)', 'Zahlen, Buchen, Netten, Margins', 'FSA', 'EMIR', 'Basel II', 'Basel III', 'COREP', 'Dodd Frank Act', 'FINREP', 'MiFID II ', 'REMIT', 'Transaction Tax', 'Standardsoftware Capital Markets', 'Bloomberg', 'Reuters', 'DTCC', 'Eurex', 'MarkitWire', 'Xetra', 'Calypso', 'Front Arena', 'Front Arena PRIME', 'Kondor+', 'Kondor Global Risk', 'Murex', 'Murex MxML', 'Openlink Endur', 'Summit', 'Rolfe & Nolan', 'Nostro Cash Manager', 'Steria Liquidity Suite', 'WST (Coba)', 'GDS (Deutsche Bank)', 'OBS', 'Capital Markets Technologien', 'FpML', 'SWIFT Interact', 'Backup & Restore', 'IAM - Activity Logging & Monitoring', 'MS Reporting Services', 'Verbraucherschutz', ' Preisangabenverordnung (PAngV)', 'E-Commerce', 'Online Konzeption', 'Suchmaschinenoptimierung', 'Content Marketing ', 'Email-Marketing', 'Social Media Marketing', 'Usability', 'AB-Testing', 'Landingpage Optimierung', 'Online-Marketing', 'Online-Strategie', 'Conversion Optimierung', 'Risikomanagement', 'Solvency II', 'Säule I', 'Säule II', 'Säule III', 'MaRisk VA', 'ORSA', 'IS-U - IDEX (GC - Common Layer) ', 'IS-U - IDEX (GENF) ', 'IS-U - UCES', 'IS-U - BPEM', 'SAP CRM Application Enhancement Tool (AET)', 'SAP CRM 5.0 / PCUI', 'SAP CRM 7.0 - Utilities Prozesse Kundenservice Privatkunden', 'SAP CRM 7.0 - Utilities Prozesse Vertrieb Geschäftskunden', 'RT.easy fachlich', 'iGrafx Process', 'Bundesagentur für Arbeit', 'Jobcenter', 'Sozialleistungsverwaltung', 'Ruby', 'Scala', 'Groovy', 'Clojure', 'Play Framework', 'Vaadin 6', 'Apache Velocity', 'Ruby on Rails', 'Grails', 'jQuery', 'AngularJS', 'IntelliJ IDEA', 'Glassfish 3', 'Dojo Toolkit', 'DokuWiki', 'CouchDB', 'ViFlow', 'Git', 'HP Service Manager V7.x', 'HP Service Manager V9.x', 'CaptainCasa Framework', 'Barrierefreiheit', 'Windows', 'HTML', 'Java Swing', 'Standardsoftware JAWS', 'Standardsoftware LUNAR', 'Elektronische Signatur ', 'Open Limit', 'Levigo Jadice Viewer', 'msgGillardon CASHVER', 'msgGillardon GuV-Planer', 'msgGillardon sDIS+', 'msgGillardon VARAN', 'msgGillardon JAdmin', 'msgGillardon MARZIPAN', 'msgGillardon RRAdmin', 'Innovationsmanagement', 'Design Thinking', 'JEE7', 'EJB 3', 'JSF 2', 'JAX-RS', 'JPA', 'CDI', 'JMS', 'JCA', 'JTA', 'Bean Validation', 'Annotations', 'Java Batch', 'WebSocket', 'Concurrency', 'Servlet', 'JSON', 'Frameworks', 'Apache Axis2', 'Apache CXF', 'JAX-WS ', 'PrimeFaces', 'Enterprise Service Bus (ESB)', 'EAI', 'Apache StormSpark', 'Apache Camel', 'Spring Integration', 'Inubit Bosch', 'Portal Plattform ', 'IBM Portal', 'Liferay Portal', 'JBoss Portal', 'Adobe Experience Manager', 'Big Data', 'Apache Cassandra', 'Apache Hadoop', 'Hortonworks Data Platform ', 'Datameer', 'Mobile', 'HTML5 Mobile Web App Framework', 'Apache Cordova', 'PhoneGap', 'Ionicframework ', 'Bootstrap', 'Sass', 'Vaadin TouchKit', 'AngularJS', 'Sencha Touch', 'jQuery Mobile', 'Business Intelligence und Analytics Platforms', 'Pentaho', 'SAS', 'Informatica', 'Talend', 'Tableau', 'Cloud', 'Amazon Web Services', 'Google Cloud', 'Oracle Cloud', 'Apple iCloud', 'Arquillian', 'SoapUI', 'Microsoft Azure', 'Axure Pro 7.0', 'Balsamiq Mockups', 'Inkscape', 'Technologien', 'Beacon', 'Bluetooth Low Energy (LE)', 'Windows 7', 'Windows 8 / 8.1', 'Windows 2008', 'Windows 2012', 'Oracle Text', 'Oracle Parallelverarbeitung & Pipelined Funktionen', 'Oracle XML-DB', 'Silverlight', 'Windows Communication Foundation (WCF)', 'PowerShell', 'Visual Studio Tools for Office (VSTO)', 'Windows Communication Foundation (WCF)', 'Success Factors Introduction', 'Success Factors Recruiting Marketing', 'Success Factors Recruiting Management', 'Success Factors Recruiting onboarding', 'Success Factors Succession Management', 'Success Factors Learning', 'Success Factors Compensation', 'Success Factors Variable Pay ', 'Success Factors Employee Central', 'Success Factors Performance & Goals', 'Success Factors Workforce analytics', 'Success Factors Employee central payroll', 'HR Renewal', 'SAP Fiori HCM', 'ESS / MSS', 'HCM Datenlöschkonzepte', 'HCM Berechtigungskonzept ', 'SAP Netweaver Business Client (NWBC)', 'BPMN', 'Winshuttle Composer', 'Winshuttle Designer', 'Winshuttle', 'Transaction', 'Direct', 'Query', 'Winshuttle', 'Winshuttle Central', 'Mediation und Konfliktmanagement', 'Tributum Evidence Distributor', 'Tributum Financial Instruments', 'Tributum Tax Engine', 'Tributum Tax Reporting', 'Job Scheduler', 'Automic UC4', 'CRON', 'IBM Tivoli Workload Scheduler (ehem. OPC)', 'BIRT', 'Business Objects Dashboards', 'Design Studio', 'Information Design Tool', 'SAP Predictive Analytics', 'Cloud Consulting', 'Cloud Standards', 'Develop Customer Cloud Strategy', 'Advise Cloud Security', 'Cloud Sourcing', 'Adopt Public Cloud', 'Assess cloud & SaaS opportunity', 'Manage Cloud Programme', 'Market your cloud', 'SaaS in the Cloud', 'Cloud / SaaS IT Architecture', 'SaaS Security Management', 'Manage SaaS Dev Programme', 'Develop SaaS Application', 'Market SaaS solutions cloud', 'Manage Cloud Services', 'Define Cloud Architecture', 'BUILD the Cloud', 'Cloud Services Management', 'RUN the cloud - General', 'RUN the Cloud - System Monitoring', 'RUN  the Cloud - IT Service Management', 'RUN the cloud - Change Management', 'RUN the Cloud - Network', 'SECURE the Cloud', 'Public Clouds', 'IBM Cloud (Bluemix & Softlayer)', 'IBM Softlayer', 'SAP Cloud', 'Juniper Networks', 'Palo Alto Networks', 'Fortinet', 'IBM Tivoli Netcool Performance Manager for Wireless (TNPMW)', 'IBM Tivoli Netcool Performance Manager for Wireless Delevopment Kit (TPDK)', 'Mycom OSI NIMS PrOptima Service and Network Performance Management', 'SSW TELCO-OSS & Security', 'Mycom OSI NIMS PrOptima Service and Network Performance Management', 'Methoden', 'MapReduce', 'Apache Frameworks', 'Apache Hbase', 'Apache Hive', 'Apache Pig', 'Apache Zookeeper', 'Spark', 'Methoden', 'Plattform / Tool', 'Qlikview', 'SAP Lumira', 'SAP BO Design Studio', 'Tibco Spotfire', 'Sprachen', 'R', 'S', 'Python', 'SAP Architektur', 'Hadoop/Hortonworks/Cloudera', 'Oracle Data Modeler', 'Oracle SQL Developer', 'PL/SQL Developer', 'SAP S/4 Hana Finance - Implementierung und Modellierung', 'SAP S/4 Hana Finance - Migration', 'LoadUI', 'Ready! API', 'Digitale Exzellenz', 'Digital Leadership', 
              'Digital Empowerment', 'Digital Platform Management', 'Process Digitisation and Automation', 'Business Model Innovation', 'IT Architecture Transformation', 'Customer and Partner Engagement', 'Data-Driven Agility', 'Digital Compliance', 'Digital Security', 'XCOM (Tristan Bankrechner)', 'CGI (Hotscan)', 'Design Thinking ', 'Graphic Recording ', 'M_o_R ', 'Openshift', 'Openstack', 'Industrie 4.0', 'Internet of Things', 'Microservices', 'ArangoDB', 'Cassandra', 'CockroachDB', 'HBase', 'InfluxDB', 'MemSQL', 'MongoDB', 'Neo4j', 'OrientDB', 'RethinkDB', 'Data Science', 'Dokumentation', 'Benutzerdokumentation', 'Entwicklerdokumentation', 'Lastenheft', 'Pflichtenheft', 'Product Backlog', 'Product Increment', 'Sprint Backlog', 'Android Studio', 'Aptana Studio', 'Atom', 'Cloud9', 'WebIDE', 'Notepad++', 'PhpStorm', 'Sublime Text', 'WebStorm', 'Xamarin Studio', 'Xcode', 'Open API', 'Angular 2', 'Apache Wicket', 'Backbone.js', 'Google Web Toolkit', 'Kendo', 'Laravel', 'Node.js', 'Postman', 'pure.css', 'ReactJS', 'SASS', 'Sencha', 'Symfony', 'Thymeleaf', 'Vaadin 7', 'Zend Framework', 'Xamarin', 'Bootstrap', 'Delphi', 'Go', 'Objective-C', 'Swift', 'Typescript', 'OmniGraffle', 'Projectplace', 'Trac', 'Trello', 'Angular CLI', 'Container ', 'DiffMerge', 'Docker', 'Gradle', 'Jenkins', 'Kubernetes', 'OpenShift', 'SourceTree', 'Amazon Alexa', 'Artificial intelligence', 'Deep learning', 'Google Assistant', 'Machine learning', 'Microsoft Cortana', 'IBM Watson', 'DevOps', 'Extreme Programming (XP)', 'Kanban', 'Spiralmodell', 'Wasserfallmodell', 'Windows 10', 'ReactiveX', 'RxJava', 'EclipseLink', 'Security', 'Java Platform Security', 'JAAS', 'JSSE', 'PKI', 'Reponsive Web Design', 'Media Query', 'StroogLoop', 'Web Components', 'Java/Dalvik', 'Native App', 'Web App', 'NativeScript', 'ngCordova', 'Datenbanktypen', 'Relational Databases', 'Graph Databases', 'Time Series Databases', 'In Memory', 'Datenbankoptimierung', 'Instanz-Tuning', 'Abfrage-Tuning', 'Blockchain', 'Plattform/Ethereum', 'Plattform/Chain', 'Plattform/IBM Blockchain', 'Plattform/Microsoft Azure', 'Plattform/R3 Corda', 'Technologie/Ethereum J', 'Umsetzung (Klassisch)', 'Systemintegrationstest', 'Whitebox-Test', 'Blackbox-Test', 'EJB3Unit', 'RPA (Robotic Prozess Automatisierung)', 'Blue Prism', 'UX-Design ', 'User Experience (UX)', 'User Research', 'User Interface Design', 'Rapid Prototyping', 'Usability Testing', 'Wireframing', 'Visual Screen Design', 'Userflows', 'Klick-Dummies', 'Design Thinking', 'HiScout ', 'Check_MK', 'Elastic Stack', 'BIC Design', 'K2 Workflows', 'K2 Smartforms', 'K2 SmartObjects', 'XÖV', 'XJustiz', 'XKfz', 'XPolizei', 'Adobe CC', 'Sketch', 'Unity', 'Private Lebensversicherung', 'Vollmachten', 'Vertriebswege', 'Vertrieb', 'Vergütung', 'Schaden/Leistung', 'Produkt', 'Partner', 'POS', 'Leistungsbearbeitung', 'Inkasso/Exkasso', 'Bestand', 'Apps', 'Swift', 'Native App', 'Web App', 'Apache Cordova', 'phone GAP', 'NativeScript', 'ngCordova', 'SQL', 'PL/SQL', 'EU: Datenschutz-Grundverordnung (DSGVO)', 'After-Sales', 'Statische Analyse', 'QA-C / QA-C++', 'McCabe', 'CodeCheck', 'Polyspace Bug Finder', 'Agiles Testen', 'Exploratives Testen', 'OO-Testen', 'Cantata / Cantata++', 'CppUnit', 'VectorCast', 'GNU GCOV', 'Normen', 'ISO/IEC 9899', 'MISRA-C/C++', 'Komodo IDE', 'LiClipse', 'Visual Basic for Applications (VBA)', 'Artisan Studio', 'SailPoint Identity IQ', 'API\xa0 - Application Programming Interface', 'REST - Representational State Transfer ', 'OAuth', 'WS-Security', 'FTP/S', 'Message Queues', 'synchrone Kommunikation', 'asynchrone Kommunikation', 'Code Review', 'Peer Review', 'Vier-Augen-Prinzip', 'Entwicklertest', 'Modultest', 'Salesforce fachlich ', 'Innovationsmanagement', 'Eltviller Modell', 'Business Model Canvas', 'ILS Software', 'OmegaPS', 'Opus10', 'LBase', 'Windchill/Relex', 'MIL-STD', '1388-2B', '1388-2A', '1629A', '882E', '1390D', 'IEC', '61508', 'ASD', 'S1000D', 'S2000M', 'S3000L', 'S4000P', 'S5000F', 'S6000T', 'SX000i', 'Doctrine (PHP)', 'SQLite ', 'Lua', 'JavaFX', 'Linux (Debian)', 'Linux (Redhat / Centos)', 'Linux (SuSe)', 'Windows 10', 'Cisco (ASA)', 'McAfee (MFE)', 'IBM (Datapower / IDG)', 'Trendmicro (Officescan / WFBS)', 'Trendmicro (IMSVA)', 'Trendmicro (IWSVA)', 'OTRS', 'CACTI', 'Postfix', 'Systemsicherheit Linux', 'Heartbeat', 'Sicherheitstechnologien ', 'IAM - Identity Management', 'IAM - User & Account Management', 'IAM - Privileged Identity and Access Management', 'IAM - Password Management & Password Self Services', 'IAM - Strong Authentification', 'IAM - Entitlement Management', 'IAM - Risiko & Kontrollmodell', 'IAM - Rule-based Controls & Control Automation', 'IAM - fachliche Konzeption & Policy Management', 'IAM - Least Privilege / Need to Know', 'IAM - Segregation of Duties (SoD)', 'IAM - Permission Delegation (temporäre Vertretungsregelung)', 'IAM - Reporting', 'IAM - Password Credential and Identity Attribute Management', 'IAM - IAM & Provisioning System Management', 'IAM - Interface Management', 'IAM - Password Management & Password Self Services', 'ServiceNow', 'LEGO® SERIOUS PLAY®', 'AppDynamics', 'SOLVARA', 'AVERA', 'INFINA', 'Cyril', 'Information Retrieval', 'Elasticsearch', 'Salesforce', 'Salesforce Sales Cloud', 'Salesforce Service Cloud', 'Salesforce Marketing Cloud', 'Salesforce Financial Service Cloud', 'Microsoft Dynamics NAV', 'Microsoft Dynamics 365', 'Microsoft Dynamics Business Central', 'EWM 9.4', 'OS-/DB Migration', 'FOSS', 'Design Patterns', 'bash', 'AMDP Script', 'Process Mining', 'Camunda BPM', 'Salesforce', 'Salesforce Administrator', 'Salesforce Developer', 'Salesforce Architect', 'Warehouse Management', 'EWM: Extended Warehouse Management', 'EWM-MFS: Extended Warehouse Management - MFS', 'New Work', 'Windows Server 2016 ', 'Windows Server 2019 ', 'Microsoft 365 / Office 365 ', 'Exchange Online ', 'Power Automate ', 'Power Apps ', 'OneDrive', 'SharePoint Online ', 'Teams', 'Yammer', 'Office 365 ', 'Intune / Enterprise Mobility ', 'Security / Compliance ', 'NoSQL', 'ISTQB ', 'Obsolescence Management', 'IEC 62402', 'BSI 200-x', 'Auditierung nach Datenschutz', 'SAVe 5.x', 'Geheimschutz', 'Branchenspezifischer Sicherheitsstandard (B3S)', 'Datenschutz Management System (DSMS)', 'Accountmanagement', 'Partnermanagement', 'Vertragsverhandlung', 'Adobe Forms', 'OData', 'Fiori/UI5', 'SAP ERP', 'SAP Berechtigungs', 'SAP Berechtigungsmanagement', 'SAP Usermanagement', 'IFRS16 Berechtigungskonzept', 'SAP-Basis', 'SAP Berechtigungskonzepte - Erstellung und Wartung/Service', 'SAP Testmanagement', 'SAP ChaRM', 'SAP Transportmanagement', 'SAP Druckerpflege', 'ITSM ', 'Diversity Management', 'Salesforce Plattform', 'Scaled Agile Framework (SAFe)', 'Product Owner', 'Salesforce-Training', 'NB-I', 'NB-IoT', 'Windows Virtual Desktop', 'JavaDataObject', 'Netzwerk - Load-Balancer', 'MagicDraw', 'Digital Enablement Platform', 'Digital Enablement Platform', 'SAP Data Intelligence', 'Innershift', 'Innersource', 'dbCRISP', 'Ansible', 'Anti Financial Crime', 'Hyperledger Fabric', 'NGINX', 'Visual Studio Code', 'Smart Cities', 'SAP BA AFI', 'SAP FPSL', 'BI-Architektur', 'BI-Organisation', 'Programmleitung', 'IT-Consulting', 'Transitionsmanagement', 'Geschäftsanbahnung', 'Förderbanken', 'Aufsichtsrat', 'Produktmanagement', 'Finanzcontrolling', 'OpRisk', 'Softwareauswahl', 'WordPress', 'Shopware', 'Hybris', 'Privatbanken', 'CRM Audit', 'Wholesale', 'B2B Vertrieb', 'Signavio ', 'FreeRTOS', 'Dari', 'LPWAN', 'wM-Bus', 'KNIME', 'COBIT', 'IT-Controlling', 'Demand Management', 'nintex', 'ShareGate', 'SharePoint 2010', 'SharePoint 2016', 'SAP SuccessFactors', 'SAP HCM', 'SAP Concur', 'SharePoint 2013', 'ADFS', 'Azure DevOps', 'SharePoint 2019', 'LCR', 'Meldewesen - AMM', 'Meldewesen - LCR', 'E-Akte', 'Organisationstalent', 'verinice', 'Bund-Länder Kooperation', 'Azure AD Connect', 'Azure Application Proxy', 'Gitlab', 'Kali Linux', 'PyCharm', 'CodeBlocks', 'Wireshark', 'Nmap', 'Virtualbox', 'MariaDB', 'Bash Bunny', 'Rubber Ducky', 'Wifi Pineapple', 'Metasploit', 'OpenVAS', 'Maltego', 'Hashcat', 'John the Ripper', 'Windows Registry', 'Organisationstheorie', 'Public Private Partnership', 'Netzwerk-Management', 'Social Engineering', 'Latex', 'Durchführung von Trainings', 'Netzwerk - Schwachstellenanalyse', 'ExtJS', 'Solid Works', 'Agile Methoden', 'scrum ', 'Objektorientierte Entwicklung', 'Polizeiliche Vorgangsverwaltung', 'Business Rule Frame', 'BRF+', 'BRFplus', 'SAP Forms by Adobe', 'SAP interactive Forms by Adobe', 'Business Rule Framework Plus', 'Oracle Exadata', 'Polizeiliche Vorgangsbearbeitung', 'Iterative Softwareentwicklung', 'Agile Softwareentwicklung']
# train_b = fetch_20newsgroups(subset='train',
#    categories=categories, shuffle=True, random_state=42)
# test_b = fetch_20newsgroups(subset='test',
#    categories=categories, shuffle=True, random_state=42)

# print('size of training set: %s' % (len(train_b['data'])))
# print('size of validation set: %s' % (len(test_b['data'])))
# print('classes: %s' % (train_b.target_names))

#x_train = ["Tätigkeit: Coaching Kundengröße: ca. 18.000 Mitarbeiter Branche: Industrie Prozesse - Personalentwicklung Aktivitäten - Konzeption Besonderheiten - Keine Eingesetzte Software - SAP R/3 HR Release 4.6B Tätigkeit: Projektmitarbeit Kundengröße: ca. 14.200 Branche: Versicherungen Prozesse - Administration - Abrechnung- Veranstaltungsmanagement Aktivitäten - Konzeption - Umsetzung Prozesse - Realisierung - Test Besonderheiten - Integration in eine bestehende SAP HR-Landschaft Besonderheiten - Keine Eingesetzte Software - SAP R/3 HR Release 4.5B Tätigkeit: Teilprojektleiter Kundengröße: ca. 150.000 Mitarbeiter Branche: Automotiv Prozesse - Administration - FI/CO-Schnittstelle - Organisationsmanagement - Veranstaltungsmanagement Aktivitäten - Konzeption - Umsetzung Prozesse - Realisierung - Programmierung - Test - Schulung Besonderheiten - Konzernweite Einführung für alle Standorte in Deutschland Eingesetzte Software - SAP R/3 HR Release 4.6C Tätigkeit: Projektmitarbeit Kundengröße: ca.4.500 Mitarbeiter Branche: Industrie Prozesse- Veranstaltungsmanagement Aktivitäten - Konzeption - Umsetzung Prozesse - Realisierung- Test Besonderheiten - Keine Eingesetzte Software - SAP R/3 HR Release 4.0B Tätigkeit: Projektmitarbeit Kundengröße: ca. 5.000 Mitarbeiter Branche: Industrie Prozesse - Organisationsmanagement Aktivitäten - Konzeption - Umsetzung Prozesse - Realisierung - Test - Schulung Besonderheiten - Keine Eingesetzte Software - SAP R/3 HR Release 4.0B Tätigkeit: Projektmitarbeit Kundengröße: ca. 5.000 Mitarbeiter Branche: Industrie Prozesse - Administration - Abrechnung - FI/CO-Schnittstelle - Organisationsmanagement - Personalkostenplanung Aktivitäten - Konzeption - Umsetzung Prozesse - Realisierung - Test Besonderheiten - Überleitung in diverse FI/CO-Systeme Eingesetzte Software - SAP R/3 HR Release 4.0B Tätigkeit: Projektmitarbeit Kundengröße: ca. 9.500 Mitarbeiter Branche: Automotiv Prozesse - Zeitwirtschaft Aktivitäten - Realisierung - Test Besonderheiten - Keine Eingesetzte Software - SAP R/3 HR Release 3.1I ,soll klappen Tätigkeit: Beratung und Leitung Fachkonzept, Realisierung und Test sowie Going-Live Entwicklungsumgebung: SAP HCM 4.7"]
# y_train = [1058720,1058773,1058810,1058811,1058315,1058330,1059276,1059321,1059322,
#  1059617,1059713,1059756,1059659,1059682,1060056,1060067,1060085 ,1060134,
#  1060152,1060411,1060413,1060414,1060416]
# x_test = ["Tätigkeit: Sachbearbeiterin Entgeltabrechnung -Betreuung der Mitarbeiter im Außendienst -Pflege Personalstammdaten Mitarbeiter -Durchführung der Abrechnungsprozesse -Kontenabstimmung mit Rechnungswesen Tätigkeit: Business Analyst Payroll -Durchführung und Koordination aller Abrechnungsprozesse inkl. aller Folgeläufe-Support der Payroll Superuser -Konfiguration und Maintenance der Payroll für alle Standorte in Deutschland und Österreich -Mitarbeit bei der Implementierung der HR-Prozesse auf Basis von SAP/R3 im Bereich HR in Deutschland und Österreich für das Modul Payroll -Begleitung Outsourcing Projekt der HR-Prozesse PA/PY -Überwachung und Implementierung von gesetzlichen Änderungen in den  Bereichen Lohnsteuer und Sozialversicherung in Deutschland -Pflege der SAP-Rollen/Berechtigungen im Bereich Payroll,Wartung und Weiterentwicklung SAP HCM Tätigkeit: Spezialist SAP HCM + Schwerpunkt SAP HCM Personalabrechnung -Support SAP Personaladministration und -abrechnung -Konfiguration SAP Personalabrechnung-Durchführung Jahreswechselaktivitäten SAP-HCM -Nacharbeiten/Testen von Support Package Einspielungen -Analyse der SAP-Hinweise,Wartung und Weiterentwicklung SAP HCM Tätigkeit: Spezialist SAP HCM + Schwerpunkt  SAP HCM Personalabrechnung -Support SAP Personaladministration/-abrechnung sowie Reisekosten -Qualifizierte Hotline -Pflege Berechtigungsrollen -Pflege Arbeitszeitpläne Tätigkeit: Unterstützung HCM First Level Service Support des FLS bei der Ticketbearbeitung für die SAP-Module Personaladministration und -Abrechnung ,Wartung und Weiterentwicklung SAP HCM Tätigkeit: Spezialist SAP HCM + Schwerpunkt SAP HCM Personalabrechnung -Support SAP Personaladministration/-Abrechnung -Qualifizierte Hotline -Pflege Berechtigungsrollen,Wartung und Weiterentwicklung SAP HCM Tätigkeit: Spezialist SAP HCM + Schwerpunkt SAP HCM PersonalabrechnungBetreuung und Weiterentwicklung in den SAP HCM Komponenten Personaladministration/-Abrechnung sowie Pflege der Berechtigungsrollen Durchführung des monatlichen Entgeltabrechnungsprozesses (Business Process Outsourcing)"]
# y_test = [1060006, 1059224,1060648,1060124,7575710,1058719,1058330]

In [15]:
# x_train = np.array(df.iloc[0:2].project_description)
# x_test = np.array(df.iloc[-1].project_description)
# y_train = np.array(df.iloc[0:2].tag_name)
# y_test = df.iloc[-1].tag_name

# print('shape', y_test.shape)
# print(y_test)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.project_description.astype(str),df.tags.astype(str), 
                                                            random_state=2018, test_size=0.1)
print(y_test.shape)

(2522,)


In [16]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
Y_train = vectorizer.fit_transform(y_train)
Y_test = vectorizer.fit_transform(y_test)
print(vectorizer.get_feature_names())

print(Y_train.toarray())

['10', '100', '10g', '11', '11g', '11i', '123', '13335', '1388', '1390d', '15', '15022', '15408', '1629a', '1990', '1999', '1x', '200', '2000', '2001', '2002', '2003', '2005', '2006', '2007', '2008', '2012', '2013', '2016', '2019', '203', '21', '22', '22301', '25', '2580', '2700x', '2a', '2b', '365', '390', '3com', '40', '400', '43', '46', '4gl', '509', '61508', '62402', '7799', '802', '881', '882e', '8i', '90', '9000', '95', '9899', '9i', 'aa', 'ab', 'abacus', 'abakus', 'abap', 'abc', 'abcd', 'abfall', 'abfrage', 'abhã', 'ablaufbeschreibung', 'ablauforganisation', 'ablauftools', 'ablesesteuerung', 'ablesung', 'abnahmetest', 'abrechnung', 'abrechnungen', 'abrechnungsprozesse', 'abrechnungssteuerung', 'absatzplanung', 'abstimmung', 'abwicklung', 'accelerated', 'accelerator', 'acceptance', 'access', 'account', 'accounting', 'accountplanung', 'acd', 'acq', 'acquiring', 'acquisitions', 'act', 'acta', 'actimize', 'actions', 'active', 'activity', 'actuate', 'ada', 'adabas', 'adapt', 'add', '

## STEP 1:  Load and Preprocess the Data
Preprocess the data using the `texts_from_array function` (since the data resides in an array).
If your documents are stored in folders or a CSV file you can use the `texts_from_folder` or `texts_from_csv` functions, respectively.

In [17]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       class_names=categories,
                                                                       preprocess_mode='bert',
                                                                       maxlen=512)
                                                                       #max_features=35000)

task: text classification
preprocessing train...
language: de


KeyError: ignored

## STEP 2:  Load the BERT Model and Instantiate a Learner object

In [ ]:
# you can disregard the deprecation warnings arising from using Keras 2.2.4 with TensorFlow 1.14.
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)

In [ ]:
learner.lr_plot()

## STEP 3: Train the Model

We train using one of the three learning rates recommended in the BERT paper: *5e-5*, *3e-5*, or *2e-5*.
Alternatively, the ktrain Learning Rate Finder can be used to find a good learning rate by invoking `learner.lr_find()` and `learner.lr_plot()`, prior to training.
The `learner.fit_onecycle` method employs a [1cycle learning rate policy](https://arxiv.org/pdf/1803.09820.pdf).



In [ ]:
learner.fit_onecycle(2e-5, 4)

We can use the `learner.validate` method to test our model against the validation set.
As we can see, BERT achieves a **96%** accuracy, which is quite a bit higher than the 91% accuracy achieved by SVM in the [scikit-learn tutorial](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html).

In [ ]:
learner.validate(val_data=(x_test, y_test), class_names=train_b.target_names)

## How to Use Our Trained BERT Model

We can call the `learner.get_predictor` method to obtain a Predictor object capable of making predictions on new raw data.

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.get_classes()

In [ ]:
predictor.predict(test_b.data[0:1])

In [ ]:
# we can visually verify that our prediction of 'sci.med' for this document is correct
print(test_b.data[0])

In [ ]:
# we predicted the correct label
print(test_b.target_names[test_b.target[0]])

The `predictor.save` and `ktrain.load_predictor` methods can be used to save the Predictor object to disk and reload it at a later time to make predictions on new data.

In [ ]:
# let's save the predictor for later use
predictor.save('/tmp/my_predictor')

In [ ]:
# reload the predictor
reloaded_predictor = ktrain.load_predictor('/tmp/my_predictor')

In [ ]:
# make a prediction on the same document to verify it still works
reloaded_predictor.predict(test_b.data[0:1])